### ¿Qué hace este script?

#### Para cada AP:
* usuarios.usuariosConectados(corresponde al total de conexiones), usuarios.sesiones_Usuarios (Corresponde al total de usuarios)
* Información detallada de los dispositivos conectados en el rango de fecha procesado: usuarios.dispositivo.mac, usuarios.dispositivo.tipo, usuarios.dispositivo.marca, usuarios.dispositivo.tecnologia, usuarios.dispositivo.sisOperativo.

#### Para cada sitio: 
* Totales por características de dispositivos: usuarios.sistemaOperativoUsuarios, usuarios.tipoDispositivoUsuarios, usuarios.marcaTerminal, usuarios.tecnologiaTerminal, usuarios.totales.dispositivos (Este ultimo guarda el total)
* usuarios.usoServicioInternetSitio es la suma en Gb consumido discriminado por usuarios.detallesTecnologiasTerminales (Bandas 2.4 o 5 GHz)
* usuarios.conteoDispositivos el cual indica el total de dispositivos conectados (Es lo mismo que total de conexiones)
* usuarios.usuariosNuevos (Esta tenía antes Nuevos y Recurrentes, pero ahora solo tiene Nuevos por solicitud de BI)
* usuarios.totales.usuariosNuevos, tiene el total de usuarios nuevos (el script por el momento solo totaliza nuevos siempre y cuando no se corra historicos)

In [1]:
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import parametros
import re
import random
import time

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [2]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
)

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy es usada para concatenar al nombre del indice principal previa inserción

In [3]:
now = datetime.now()
fecha_hoy = str(now.strftime("%Y.%m.%d"))

### nombre de indice donde se insertará

In [4]:
indice = parametros.usuarios_tablero12_index
indice_control = parametros.tableros_mintic_control

### Funcion para construir JSON compatible con ElasticSearch

In [5]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

In [6]:
total_docs = 1
try:
    response = es.search(
        index= indice_control,
        body={
               "_source": ['usuarios.fechaControl'],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia-tablero12"
                    }
                  }
                  ]
                }
              }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['usuarios.fechaControl']
except:
    fecha_ejecucion = '2021-05-01 00:00:00'
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-05-01 00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion)

ultima fecha para control de ejecucion: 2021-06-01 00:00:00


### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:
* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede, energía, latitud, longitud,COD_ISO, id_Beneficiario).

In [7]:
t1=time.time()
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob', 'nombreSede' 
                           , 'energiadesc', 'latitud', 'longitud', 'COD_ISO','id_Beneficiario']
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
#     fields = {}
#     for num, doc in enumerate(elastic_docs):
#         source_data = doc["_source"]
#         for key, val in source_data.items():
#             try:
#                 fields[key] = np.append(fields[key], val)
#             except KeyError:
#                 fields[key] = np.array([val])

#     datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
    
    datos_semilla = pd.DataFrame([x["_source"] for x in elastic_docs])
    
except:
    exit()
    
t2=time.time()

In [8]:
datos_semilla

,nombreSede,longitud,latitud,COD_ISO,nombre_centro_pob,site_id,nombre_departamento,energiadesc,nombre_municipio,id_Beneficiario
0,C.E.R LA PAULINA,"-76,14525469","6,7048548",CO-ANT,LA PAULINA,22154-ZGYO176,ANTIOQUIA,RED INTERCONECTADA,VALDIVIA,22154
1,CENTRO EDUCATIVO RURAL EL CERRO,"-73,10716667","4,225166667",CO-ANT,EL CERRO,20022-ZGYO177,ANTIOQUIA,RED INTERCONECTADA,FRONTINO,20022
2,C. E. R. MERCEDES YEPES,"-75,151177190","6,93848274",CO-ANT,VDA. LA MESETA,18787-,ANTIOQUIA,RED INTERCONECTADA,ANORÍ,18787
3,C. E. R. MEDIAS FALDAS,"-75,19442611","7,04797951",CO-ANT,VDA EL LIMÓN,18802-,ANTIOQUIA,TIENE ENERGÍA,ANORÍ,18802
4,C. E. R. ROSAURA ORTEGA,"-75,22730972","7,06272584",CO-ANT,BELLAVISTA,18804-ZGYO583,ANTIOQUIA,RED INTERCONECTADA,ANORÍ,18804
...,...,...,...,...,...,...,...,...,...,...
6942,ESCUELA SANTA ISABEL,"-73,138349330","6,39026335",CO-VAU,COMUNIDAD SANTA ISABEL,67733-,VAUPÉS,SOLUCIÓN SOLAR,PACOA,67733
6943,ESCUELA RURAL DE SAN LUIS DE PIEDRA ÑI,"-70,07416384","0,08164385",CO-VAU,COMUNIDAD DE SAN LUIS,70622-,VAUPÉS,SIN INFORMACIÓN,PACOA,70622
6944,ESCUELA RURAL DE SANTA ROSA,,,CO-VAU,COMUNIDAD DE SANTA ROSA,70623-,VAUPÉS,SIN INFORMACIÓN,PACOA,70623
6945,ESCUELA RURAL DE TOAKA,"-71,25103723","1,007644476",CO-VAU,COMUNIDAD DE TOAKA,70624-,VAUPÉS,SIN INFORMACIÓN,PACOA,70624


### Cambiando nombre de campos y generando location

* Se valida latitud y longitud. Luego se calcula campo location
* Se renombran los campos de semilla

In [9]:
def get_location(x):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return 'a'
datos_semilla['latitud'] = datos_semilla['latitud'].apply(get_location)
datos_semilla['longitud'] = datos_semilla['longitud'].apply(get_location)
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["longitud"]=='a') | (datos_semilla["latitud"]=='a')].index)
datos_semilla['usuarios.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['usuarios.location']=datos_semilla['usuarios.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)

In [10]:
datos_semilla = datos_semilla.rename(columns={'lugar_cod' : 'usuarios.centroDigitalUsuarios'
                                            , 'nombre_municipio': 'usuarios.nombreMunicipio'
                                            , 'nombre_departamento' : 'usuarios.nombreDepartamento'
                                            , 'nombre_centro_pob': 'usuarios.localidad'
                                            , 'nombreSede' : 'usuarios.nomCentroDigital'
                                            , 'energiadesc' : 'usuarios.sistemaEnergia'
                                            , 'COD_ISO' : 'usuarios.codISO'
                                            , 'id_Beneficiario' : 'usuarios.idBeneficiario'})

Se descartan los registros que tengan la latitud y longitud vacía o no valida

In [11]:
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["usuarios.location"]=='')].index)

### leyendo indice cambium-devicedevices

De esta formas se asocia las MAC de dispositivos de red INDOOR y OUTDOOR
* site_id para cruzar con las misma llave de semilla.
* datos del dispositivo: mac, status, ip.
* ap_group para identificar si la conexión es indoor/outdoor

In [12]:
t1=time.time()
total_docs = 30000
try:
    response = es.search(
        index= parametros.cambium_d_d_index,
        body={
                    "_source": ["site_id","mac","status","ip","ap_group"]
                  , "query": {
                    "match_all": {}
                  }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
#     fields = {}
#     for num, doc in enumerate(elastic_docs):
#         source_data = doc["_source"]
#         for key, val in source_data.items():
#             try:
#                 fields[key] = np.append(fields[key], val)
#             except KeyError:
#                 fields[key] = np.array([val])

#     datos_dev = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
    
    datos_dev = pd.DataFrame([x["_source"] for x in elastic_docs])
    
    
except:
    exit()
    
t2=time.time()

Se descartan registros con site_id vacios ya que no cruzarán en el merge y se limpian los NaN del dataframe.

In [13]:
datos_dev.dropna(subset=['site_id'], inplace=True)
datos_dev.fillna('', inplace=True)
datos_dev = datos_dev.drop(datos_dev[(datos_dev['site_id']=='')].index)

In [14]:
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("-", n = 1, expand = True)[0]
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("_", n = 1, expand = True)[0]
datos_dev['ap_group'] = datos_dev['ap_group'].str.split(".", n = 1, expand = True)[0]
datos_dev = datos_dev.drop(datos_dev[(datos_dev['ap_group']=='')].index)

Se toman solo los datos unicos con mac.

In [15]:
#datos_dev.drop_duplicates(subset=['site_id','mac'],inplace=True)
datos_dev = datos_dev.drop_duplicates('mac')

Se cambia el nombre a la mac del dispositivo de red para no confundir con la de dispositivos de usuario 

In [16]:
datos_dev= datos_dev.rename(columns={'mac' : 'usuarios.macRed','ap_group' : 'usuarios.apGroup'})

In [17]:
datos_dev

,usuarios.apGroup,ip,site_id,usuarios.macRed,status
0,INDOOR,172.28.118.179,44745-ZGYO227,BC:E6:7C:5E:66:88,online
1,OUTDOOR,172.28.118.180,44745-ZGYO227,58:C1:7A:E9:C9:6F,online
2,OUTDOOR,172.28.118.181,44745-ZGYO227,58:C1:7A:E9:C8:5C,online
3,INDOOR,172.28.118.171,44747-ZGYO413,BC:E6:7C:ED:F2:4F,online
4,OUTDOOR,172.28.118.172,44747-ZGYO413,BC:E6:7C:4F:07:88,online
...,...,...,...,...,...
5058,OUTDOOR,172.25.84.76,37717-ZZZY751,BC:A9:93:0C:7B:9D,offline
5059,OUTDOOR,172.25.84.77,37717-ZZZY751,BC:A9:93:0C:E3:1D,offline
5060,INDOOR,172.25.38.243,47123-ZGYO420,BC:E6:7C:ED:8A:EA,online
5063,OUTDOOR,172.25.60.212,47847-VZVF114,BC:A9:93:00:97:B7,offline


### Lectura de datos ohmyfi-detalleconexiones

Los datos que se toman son:
* fechahora (de cada conexión). fecha_control es lo mismo pero con el intervalo de 10 minutos
* Información del centro: lugar, lugar_cod.
* Información del dispositivo de usuario: mac_usuario, dispositivo, sistema_operativo.
* Información de usuario: tipodoc y documento

In [18]:
def trae_conexiones(fecha_ini,fecha_fin):
    total_docs = 5000000
    response = es.search(
        index= parametros.ohmyfi_d_c_index,
        body={
                "_source": ["fechahora","fecha_control","lugar","lugar_cod","mac_usuario", "dispositivo"
                            ,"sistema_operativo",'tipodoc','documento']
                , "query": {
                  "range": {
                    "fechahora": {
                      "gte": fecha_ini,
                      "lt": fecha_fin
                    }
                  }
              }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
#     fields = {}
#     for num, doc in enumerate(elastic_docs):
#         source_data = doc["_source"]
#         for key, val in source_data.items():
#             try:
#                 fields[key] = np.append(fields[key], val)
#             except KeyError:
#                 fields[key] = np.array([val])

#     return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) 

    return pd.DataFrame([x["_source"] for x in elastic_docs])

### Lanzando ejecución de consulta

* Se calcula rango en base a la fecha de control. Para este caso es de 10 minutos.
* Se ejecuta la función de consulta con el rango de fechas.
* Si no retorna datos se incrementa el rango y se ejecuta nuevamente. Este proceso se repite hasta conseguir datos o hasta que el rango de ejecución alcance la fecha y hora actual.


In [19]:
fecha_max_mintic = fecha_ejecucion
fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_det_conex = trae_conexiones(fecha_max_mintic,fecha_tope_mintic)

if datos_det_conex is None or datos_det_conex.empty:
       while (datos_det_conex is None or datos_det_conex.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
            fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
            fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
            datos_det_conex = trae_conexiones(fecha_max_mintic,fecha_tope_mintic)
else:
    pass

In [20]:
datos_det_conex['lugar_cod'] = datos_det_conex['lugar_cod'].str.strip()

In [21]:
datos_det_conex['fecha'] = datos_det_conex['fecha_control'].str.split(" ", n = 1, expand = True)[0]
datos_det_conex.drop_duplicates(subset=["fecha_control","lugar","lugar_cod","mac_usuario", "dispositivo","sistema_operativo",'tipodoc','documento'],inplace=True)

In [22]:
datos_det_conex = datos_det_conex.rename(columns={'lugar_cod' : 'site_id'
                                                             ,'fechahora':'usuarios.fechaConexionUsuarios'
                                                             ,'dispositivo': 'usuarios.tipoDispositivoUsuarios'
                                                             , 'sistema_operativo': 'usuarios.sistemaOperativoUsuarios'})

Se corrigen valores errados de site_id en detalle conexiones

In [23]:
datos_det_conex['site_id'] = datos_det_conex['site_id'].str.replace("_","-")

In [24]:
datos_det_conex

,usuarios.fechaConexionUsuarios,mac_usuario,usuarios.tipoDispositivoUsuarios,lugar,documento,tipodoc,site_id,fecha_control,usuarios.sistemaOperativoUsuarios,fecha
0,2021-06-01 07:00:04,A8:C8:3A:75:F3:87,Smartphone,VDA BARZAL,13926211,CC,45953-ZGYO991,2021-06-01 07:00:00,Android,2021-06-01
1,2021-06-01 07:00:05,0C:CE:F6:6F:40:79,Smartphone,VEREDA EL CIELO CHALAN,18881869,CC,47142-ZGYO309,2021-06-01 07:00:00,Android,2021-06-01
2,2021-06-01 07:00:09,60:14:66:E3:24:27,Smartphone,"GALILEA, MONTERIA",1007909548,CC,31154-ZGYO864,2021-06-01 07:00:00,Android,2021-06-01
3,2021-06-01 07:00:14,BC:76:5E:89:41:FD,Smartphone,SAN JOSE DE CUCUTA_ORIPAYA,19400585,CC,41917-ZZZY071,2021-06-01 07:00:00,Android,2021-06-01
4,2021-06-01 07:00:14,A8:81:95:66:D1:8E,Smartphone,"CARAMELO,TIERRA ALTA",25784850,CC,32522-ZGYO882,2021-06-01 07:00:00,Android,2021-06-01
...,...,...,...,...,...,...,...,...,...,...
17120,2021-06-01 23:09:34,90:63:3B:DD:F1:B1,Smartphone,"VDA MANICA, SAN JOSE DE TOLUVIEJO",1002798622,CC,47741-ZGYO978,2021-06-01 23:00:00,Android,2021-06-01
17121,2021-06-01 23:09:34,84:B5:41:7B:FB:98,Smartphone,BROQUELES MOÑITOS,73198552,CC,31839-ZGYO456,2021-06-01 23:00:00,Android,2021-06-01
17123,2021-06-01 23:09:43,88:10:8F:FD:4D:39,Smartphone,"VDA ALTO CALOTO, PAICOL",1004082181,CC,36899-ZZZY215,2021-06-01 23:00:00,Android,2021-06-01
17125,2021-06-01 23:09:45,A8:81:95:38:AA:36,Smartphone,BARRIO EL PANTEON CALLE MANGA_SAN ANTERO,1072526730,CC,13695-ZGYO347,2021-06-01 23:00:00,Android,2021-06-01


### Se lee el indice all-cambium-device-client

* En este indice se guarda el detalle de los radio por fecha
* Detalle conexiones cruza con device clients. Con estos se calculan los totales por marca

In [25]:
def traeRadio(fecha_max2,fecha_tope2):
    total_docs = 100000
    response = es.search(
        index= 'all-'+parametros.cambium_d_c_index,
        body={
            "_source": ['mac', 'ap_mac', 'radio.band', 'radio.rx_bytes', 'radio.tx_bytes','fecha_control']
              , "query": {
                  "range": {
                    "fecha_control": {
                      "gte": fecha_max_mintic2,
                      "lt": fecha_tope_mintic2
                      #"gte": "2021-05-26 15:00:00",
                      #"lt": "2021-05-26 15:10:00"  
                    }
                  }
              }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
    # fields = {}
    # for num, doc in enumerate(elastic_docs):
    #     source_data = doc["_source"]
    #     for key, val in source_data.items():
    #         try:
    #             fields[key] = np.append(fields[key], val)
    #         except KeyError:
    #             fields[key] = np.array([val])

    # datos_radio = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))

    return pd.DataFrame([x["_source"] for x in elastic_docs])

    

In [26]:
fecha_max_mintic2 = fecha_ejecucion

fecha_tope_mintic2 = (datetime.strptime(fecha_max_mintic2, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_performance = traeRadio(fecha_max_mintic2,fecha_tope_mintic2)


if datos_performance is None or datos_performance.empty:
    while (datos_performance is None or datos_performance.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic2 = (datetime.strptime(fecha_max_mintic2, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic2 = (datetime.strptime(fecha_tope_mintic2, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
        datos_performance = traeRadio(fecha_max_mintic2,fecha_tope_mintic2)
else:
    pass

In [27]:
try:
    datos_logins = datos_det_conex[['fecha_control', 'site_id', 'documento','usuarios.macRed']].groupby(["fecha_control","site_id","usuarios.macRed"])['documento'].nunique().reset_index()
    datos_logins= datos_logins.rename(columns={'documento' : 'usuarios.sesiones_Usuarios'})
    
    ################# datos_performance #####################
    datos_performance['fecha_control'] = datos_performance["fecha_control"].str[0:-4] + '0:00'
    datos_performance.replace('','0',inplace=True)
    datos_performance.fillna({'radio.rx_bytes':0,'radio.tx_bytes':0},inplace=True)
    datos_performance['radio.rx_bytes'] = datos_performance['radio.rx_bytes'].astype(float)
    datos_performance['radio.tx_bytes'] = datos_performance['radio.tx_bytes'].astype(float)
    
    datos_performance = datos_performance.groupby(['ap_mac', 'fecha_control', 'mac', 'radio.band']).agg(['max']).reset_index()
    datos_performance.columns = datos_performance.columns.droplevel(1)
    datos_performance = datos_performance[['ap_mac'
                                         , 'fecha_control'
                                         , 'radio.rx_bytes'
                                         , 'radio.tx_bytes']].groupby(['ap_mac'
                                                                     , 'fecha_control']).agg(['sum']).reset_index()
    datos_performance.columns = datos_performance.columns.droplevel(1)
    datos_performance = datos_performance.rename(columns={'radio.rx_bytes': 'usuarios.consumoUsuariosDescarga_aux'
                                                     , 'radio.tx_bytes':'usuarios.consumoUsuariosCarga_aux'})
    
    datos_performance['usuarios.consumoUsuariosDescarga'] = round((datos_performance['usuarios.consumoUsuariosDescarga_aux']/float(1<<30)),6)
    datos_performance['usuarios.consumoUsuariosCarga'] = round((datos_performance['usuarios.consumoUsuariosCarga_aux']/float(1<<30)),6)
    datos_performance['usuarios.consumoUsuarios'] = datos_performance['usuarios.consumoUsuariosDescarga'] + datos_performance['usuarios.consumoUsuariosCarga']
    datos_performance['usuarios.consumoUsuarios'] = round(datos_performance['usuarios.consumoUsuarios'],6)
    datos_performance = datos_performance.rename(columns={'ap_mac':'usuarios.macRed'})
    
    
    datos_performance = pd.merge(datos_performance,datos_dev[['site_id','usuarios.macRed','usuarios.apGroup']],on='usuarios.macRed', how='left')
    
    aux_performance = pd.merge(datos_performance,  datos_semilla, on='site_id',how='inner')
    aux_performance.fillna({'usuarios.consumoUsuariosDescarga':0,
                              'usuarios.consumoUsuariosCarga':0,
                              'usuarios.consumoUsuarios':0,
                              'usuarios.apGroup':'No identificado'},inplace=True)
    aux_performance = aux_performance.rename(columns={'fecha_control':'usuarios.fechaControl'
                                                           ,'site_id' : 'usuarios.siteID'})
    
    aux_performance["usuarios.fecha"] = aux_performance["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    aux_performance["usuarios.anyo"] = aux_performance["usuarios.fecha"].str[0:4]
    aux_performance["usuarios.mes"] = aux_performance["usuarios.fecha"].str[5:7]
    aux_performance["usuarios.dia"] = aux_performance["usuarios.fecha"].str[8:10]
    aux_performance["usuarios.hora"] = aux_performance["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    aux_performance["usuarios.minuto"] = aux_performance["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    
    aux_performance['nombreDepartamento'] = aux_performance['usuarios.nombreDepartamento']
    aux_performance['nombreMunicipio'] = aux_performance['usuarios.nombreMunicipio']
    aux_performance['idBeneficiario'] = aux_performance['usuarios.idBeneficiario']
    aux_performance['fecha'] = aux_performance['usuarios.fecha']
    aux_performance['anyo'] = aux_performance['usuarios.anyo']
    aux_performance['mes'] = aux_performance['usuarios.mes']
    aux_performance['dia'] = aux_performance['usuarios.dia']
    
    
    ##################
    usuariosConectados = datos_det_conex[["fecha_control","site_id","usuarios.macRed","mac_usuario"]].groupby(["fecha_control","site_id","usuarios.macRed"]).agg(['count']).reset_index()
    usuariosConectados.columns = usuariosConectados.columns.droplevel(1)
    usuariosConectados= usuariosConectados.rename(columns={'mac_usuario' : 'usuarios.tablero11usuariosConectados'})
    
    ###################
    usuariosConectados = pd.merge(usuariosConectados,datos_logins, on=["fecha_control","site_id","usuarios.macRed"], how='inner')
    
    usuariosConectados= usuariosConectados.rename(columns={'usuarios.sesiones_Usuarios' : 'usuarios.tablero11sesiones_Usuarios'})
    
    ###################
    usuariosConectados = pd.merge(datos_semilla,  usuariosConectados, on=['site_id'], how='inner')
    
   
    #Al parecer no esta viniendo consumoUsuario....
    
    usuariosConectados.fillna({'usuarios.consumoUsuarios' : 0
                              ,'usuarios.consumoUsuariosDescarga':0
                              ,'usuarios.consumoUsuariosCarga':0},inplace=True)
    
    ####################
    usuariosConectados = pd.merge(usuariosConectados,datos_dev[['usuarios.apGroup','usuarios.macRed']], on ='usuarios.macRed', how='left')
    
    usuariosConectados.fillna({'usuarios.apGroup':'No identificado'},inplace=True)
    usuariosConectados['usuarios.tablero11usuariosConectados'] = usuariosConectados['usuarios.tablero11usuariosConectados'].astype(int)
    usuariosConectados = usuariosConectados.rename(columns={'fecha_control':'usuarios.fechaControl'
                                                           ,'site_id' : 'usuarios.siteID'})
    
    usuariosConectados["usuarios.fecha"] = usuariosConectados["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    usuariosConectados["usuarios.anyo"] = usuariosConectados["usuarios.fecha"].str[0:4]
    usuariosConectados["usuarios.mes"] = usuariosConectados["usuarios.fecha"].str[5:7]
    usuariosConectados["usuarios.dia"] = usuariosConectados["usuarios.fecha"].str[8:10]
    usuariosConectados["usuarios.hora"] = usuariosConectados["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    usuariosConectados["usuarios.minuto"] = usuariosConectados["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    
    usuariosConectados['nombreDepartamento'] = usuariosConectados['usuarios.nombreDepartamento']
    usuariosConectados['nombreMunicipio'] = usuariosConectados['usuarios.nombreMunicipio']
    usuariosConectados['idBeneficiario'] = usuariosConectados['usuarios.idBeneficiario']
    usuariosConectados['fecha'] = usuariosConectados['usuarios.fecha']
    usuariosConectados['anyo'] = usuariosConectados['usuarios.anyo']
    usuariosConectados['mes'] = usuariosConectados['usuarios.mes']
    usuariosConectados['dia'] = usuariosConectados['usuarios.dia']
    
    #####################

    columnas_aux_performance=['usuarios.macRed', 'usuarios.fechaControl',\
                               'usuarios.consumoUsuariosDescarga_aux',
                               'usuarios.consumoUsuariosCarga_aux', 'usuarios.consumoUsuariosDescarga',\
                               'usuarios.consumoUsuariosCarga', 'usuarios.consumoUsuarios',\
                               'usuarios.siteID', 'usuarios.apGroup']
    columnas_merge = ['usuarios.fechaControl','usuarios.siteID','usuarios.apGroup','usuarios.macRed']
    
    usuariosConectados=pd.merge(usuariosConectados,aux_performance[columnas_aux_performance],on=columnas_merge, how='left')
    

    usuariosConectados = usuariosConectados.rename(columns=
                                                   {'usuarios.consumoUsuariosCarga_aux':'usuarios.tablero11consumoUsuariosCarga_aux'
                                                    ,'usuarios.consumoUsuariosDescarga_aux':'usuarios.tablero11consumoUsuariosDescarga_aux'
                                                    ,'usuarios.consumoUsuariosCarga' : 'usuarios.tablero11consumoUsuariosCarga'
                                                    ,'usuarios.consumoUsuariosDescarga':'usuarios.tablero11consumoUsuariosDescarga'
                                                    ,'usuarios.consumoUsuarios':'usuarios.tablero11consumoUsuarios'
                                                   })

except:
    pass

In [28]:
try:
    use_these_keys = ['usuarios.tablero11.siteID'
                      ,'usuarios.tablero11.nomCentroDigital'
                      , 'usuarios.tablero11.localidad'
                      , 'usuarios.tablero11.nombreDepartamento'
                      , 'usuarios.tablero11.nombreMunicipio'
                      , 'usuarios.tablero11.idBeneficiario'
                      , 'usuarios.tablero11.location'
                      , 'usuarios.tablero11.sesiones_Usuarios'
                      , 'usuarios.tablero11.usuariosConectados'
                      , 'usuarios.tablero11.consumoUsuarios'
                      , 'usuarios.tablero11.consumoUsuariosDescarga_aux'
                      , 'usuarios.tablero11.consumoUsuariosCarga_aux'
                      , 'usuarios.tablero11.consumoUsuariosDescarga'
                      , 'usuarios.tablero11.consumoUsuariosCarga'
                      , 'usuarios.tablero11.apGroup'
                      , 'usuarios.tablero11.fechaControl'
                      , 'usuarios.tablero11.fecha'
                      , 'usuarios.tablero11.anyo'
                      , 'usuarios.tablero11.mes'
                      , 'usuarios.tablero11.macRed'
                      , '@timestamp']


    usuariosConectados['@timestamp'] = now.isoformat()
    
    #Seleccionar campos del Tablero..
    
    #Cambio de variables en usuariosConectados...
    
    usuariosConectados = cambioVariable(usuariosConectados)
    
    
    def doc_generator_consumo(df,use_these_keys):
        df_iter = df.iterrows()
        for index, document in df_iter:
            yield {
                    "_index": indice, 
                    "_id": f"{ 'usuariosConectados-'+ str(document['usuarios.tablero11.siteID']) + str(document['usuarios.tablero11.macRed']) + '-' + str(document['usuarios.tablero11.fechaControl']) +'-'+ str(random.randrange(10000))}",
                    "_source": filterKeys(document,use_these_keys),
                }

    #Se comenta escritura en el indice porque no se necesita esa estructura#Se comenta escritura en el indice porque no se necesita esa estructura            }
    
#     usuariosConectados.fillna("",inplace=True)

    usuariosConectados.fillna({'usuarios.tablero11.sesiones_Usuarios':0
                              , 'usuarios.tablero11.usuariosConectados':0
                              , 'usuarios.tablero11.consumoUsuarios':0
                              , 'usuarios.tablero11.consumoUsuariosDescarga_aux':0
                              , 'usuarios.tablero11.consumoUsuariosCarga_aux':0
                              , 'usuarios.tablero11.consumoUsuariosDescarga':0
                              , 'usuarios.tablero11.consumoUsuariosCarga':0},inplace=True)
    
    usuariosConectados.fillna("",inplace=True)
    
    salida = helpers.bulk(es, doc_generator_consumo(usuariosConectados,use_these_keys))
    
    print("Fecha: ", now,"- Usuarios conectados insertados en indice principal:",salida[0])
    
except Exception as e:  
    
    print("Fecha: ", now,"- Ningun usuario conectado para insertar en indice principal:")

Fecha:  2021-08-17 22:13:38.600373 - Ningun usuario conectado para insertar en indice principal:


Se cruza all-cambium-device-clients con cambium-devicedevices para obtener el site_id

In [29]:
datos_performance = datos_performance.rename(columns={'ap_mac':'usuarios.macRed', 'mac':'mac_usuario'})

In [30]:
usuarios_conectados_cambium = pd.merge(datos_performance,datos_dev, on ='usuarios.macRed', how='inner')

In [31]:
datos_performance

,usuarios.macRed,fecha_control,mac_usuario,radio.band,radio.rx_bytes,radio.tx_bytes
0,BC:E6:7C:58:E7:A1,2021-06-01 07:58:00,50:3D:C6:1E:FC:52,5GHz,6276,0
1,BC:E6:7C:5B:AF:F6,2021-06-01 07:58:00,D0:04:01:6B:9D:37,2.4GHz,4588,5109
2,BC:E6:7C:4E:7E:1E,2021-06-01 07:58:00,B8:94:36:BD:E9:C7,2.4GHz,232491,6830388
3,BC:E6:7C:4F:05:F1,2021-06-01 07:58:00,24:DA:9B:02:41:C0,2.4GHz,714147,5575763
4,BC:E6:7C:5B:7D:4B,2021-06-01 07:58:00,7C:2E:DD:59:DD:1A,2.4GHz,2707004,698339
...,...,...,...,...,...,...
99995,BC:E6:7C:4F:05:C9,2021-06-01 07:06:00,3C:DC:BC:8C:FA:2B,2.4GHz,2543,0
99996,58:C1:7A:E8:76:8B,2021-06-01 07:06:00,DC:BF:E9:2C:B2:E9,2.4GHz,379509,128341
99997,BC:E6:7C:5B:7C:D8,2021-06-01 07:06:00,F8:1F:32:46:60:DD,2.4GHz,28749,39868
99998,BC:E6:7C:4F:07:2F,2021-06-01 07:06:00,18:D6:1C:0E:EA:A0,2.4GHz,225096,65398


In [32]:
usuarios_conectados_cambium

,usuarios.macRed,fecha_control,mac_usuario,radio.band,radio.rx_bytes,radio.tx_bytes,usuarios.apGroup,ip,site_id,status
0,BC:E6:7C:58:E7:A1,2021-06-01 07:58:00,50:3D:C6:1E:FC:52,5GHz,6276,0,OUTDOOR,172.25.58.108,36564-ZGYO609,online
1,BC:E6:7C:58:E7:A1,2021-06-01 07:58:00,DE:41:79:1F:EE:B6,5GHz,2400,0,OUTDOOR,172.25.58.108,36564-ZGYO609,online
2,BC:E6:7C:58:E7:A1,2021-06-01 07:58:00,60:AB:67:B6:D2:94,5GHz,2911,0,OUTDOOR,172.25.58.108,36564-ZGYO609,online
3,BC:E6:7C:58:E7:A1,2021-06-01 08:00:00,DE:41:79:1F:EE:B6,5GHz,2400,0,OUTDOOR,172.25.58.108,36564-ZGYO609,online
4,BC:E6:7C:58:E7:A1,2021-06-01 08:00:00,60:AB:67:B6:D2:94,5GHz,2911,0,OUTDOOR,172.25.58.108,36564-ZGYO609,online
...,...,...,...,...,...,...,...,...,...,...
99716,BC:E6:7C:58:E7:EB,2021-06-01 06:30:00,00:27:15:02:7E:32,2.4GHz,175300,76426,OUTDOOR,172.25.22.213,31858-ZGYO283,online
99717,BC:E6:7C:58:E7:EB,2021-06-01 06:38:00,00:27:15:02:7E:32,2.4GHz,386,0,OUTDOOR,172.25.22.213,31858-ZGYO283,online
99718,BC:E6:7C:58:E7:EB,2021-06-01 06:28:00,00:27:15:02:7E:32,2.4GHz,175300,76426,OUTDOOR,172.25.22.213,31858-ZGYO283,online
99719,BC:E6:7C:58:E7:EB,2021-06-01 06:26:00,00:27:15:02:7E:32,2.4GHz,175300,76426,OUTDOOR,172.25.22.213,31858-ZGYO283,online


### Se lee la información de cambium device client

Esta lectura se usa para identificar: 
* Lecturas de consumo por dispositivo (radios rx y tx)
* Fabricante del dispositivo
* la mac del ap que luego se usa para identificar el ap group(Indoor/Outdoor)

In [33]:
t1=time.time()

total_docs = 30000000
response = es.search(
    index= parametros.cambium_d_c_index,
    body={
            "_source": ["mac","ap_mac", 'manufacturer','name',"radio.band",'radio.rx_bytes','radio.tx_bytes'],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"mac"
                    }
                  }
                  ]
                }
              }
    },
    size=total_docs
)
elastic_docs = response["hits"]["hits"]
# fields = {}
# for num, doc in enumerate(elastic_docs):
#     source_data = doc["_source"]
#     for key, val in source_data.items():
#         try:
#             fields[key] = np.append(fields[key], val)
#         except KeyError:
#             fields[key] = np.array([val])

# datos_dev_clients = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))

datos_dev_clients = pd.DataFrame([x["_source"] for x in elastic_docs])

t2=time.time()

In [34]:
(t2-t1)/60

0.7608158508936564

In [35]:
df1=datos_dev_clients

In [36]:
(t2-t1)/60

0.7608158508936564

In [37]:
df2=datos_dev_clients

In [38]:
df1.equals(df2)

True

In [39]:
df1.shape

(242677, 7)

In [40]:
df2.shape

(242677, 7)

### Asociando datos de usuarios¶

Se realiza lectura por día procesado
lugar_cod se usa para cruzar con semilla por site_id
creado es la fecha de creación del usuario
tipodoc y documento corresponden a datos del usuario. Se usa para calcular totales

In [41]:
def traeRegistros(fecha_max,fecha_tope):
    total_docs = 10000
    response = es.search(
        index= parametros.ohmyfi_d_u_index,
        body={
                "_source": ['lugar_cod','tipodoc', 'documento', 'creado']
                  , "query": {
                  "range": {
                    "creado": {
                      "gte": fecha_max.split(' ')[0]+' 00:00:00',
                      "lt": fecha_tope.split(' ')[0]+' 23:59:59'
                      #"gte": "2021-06-01 00:00:00",
                      #"lt": "2021-06-01 23:59:59"  
                    }
                  }
              }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))

### Se realiza consulta de datos

* Se calcula rango en base a la fecha de control. Para este caso es de un día.
* Se ejecuta la función de consulta con el rango de fechas.
* Si no retorna datos se incrementa el rango y se ejecuta nuevamente. Este proceso se repite hasta conseguir datos o hasta que el rango de ejecución alcance la fecha actual.

In [42]:
fecha_max_mintic = fecha_ejecucion
fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_registro = traeRegistros(fecha_max_mintic,fecha_tope_mintic)

if datos_registro is None or datos_registro.empty:
    while (datos_registro is None or datos_registro.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
        datos_speed = traeRegistros(fecha_max_mintic,fecha_tope_mintic)
else:
    pass

## Se lee información de Ohmyfi consumos

Apartir de esta lectura se toma el valor del tiempo promedio de sesión en minutos. Se toma de referencia los campos:

fecha_inicio a partir de la cual se calcula fecha control
tiempo_sesion_minutos
mac_usuario
lugar_cod

In [43]:
def traeSesiones(fecha_max,fecha_tope):
    total_docs = 500000
    response = es.search(
        index= parametros.ohmyfi_consumos_index,
        body={
                  "_source": ["lugar_cod", "tiempo_sesion_minutos","mac_ap","fecha_inicio"]
                 ,"query": {
                      "range": {
                            "fecha_inicio": {
                            "gte": fecha_max,
                            "lt": fecha_tope
                            }
                        }
                  }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
#     fields = {}
#     for num, doc in enumerate(elastic_docs):
#         source_data = doc["_source"]
#         for key, val in source_data.items():
#             try:
#                 fields[key] = np.append(fields[key], val)
#             except KeyError:
#                 fields[key] = np.array([val])

#     return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) 
    return pd.DataFrame([x["_source"] for x in elastic_docs])


In [44]:
def trae_conexiones(fecha_ini,fecha_fin):
    total_docs = 5000000
    response = es.search(
        index= parametros.ohmyfi_d_c_index,
        body={
                "_source": ["fechahora","fecha_control","lugar","lugar_cod","mac_usuario", "dispositivo"
                            ,"sistema_operativo",'tipodoc','documento']
                , "query": {
                  "range": {
                    "fechahora": {
                      "gte": fecha_ini,
                      "lt": fecha_fin
                    }
                  }
              }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]

    return pd.DataFrame([x["_source"] for x in elastic_docs])

## Se ejecuta consulta de datos

Se calcula rango en base a la fecha de control. Para este caso es de 10 minutos.
Se ejecuta la función de consulta con el rango de fechas.
Si no retorna datos se incrementa el rango y se ejecuta nuevamente. Este proceso se repite hasta conseguir datos o hasta que el rango de ejecución alcance la fecha y hora actual.

In [45]:
fecha_max_mintic = fecha_ejecucion

fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_consumos = traeSesiones(fecha_max_mintic,fecha_tope_mintic)

if datos_consumos is None or datos_consumos.empty:
    while (datos_consumos is None or datos_consumos.empty) and ((datetime.strptime(fecha_max_mintic[0:50], '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)).strftime("%Y-%m-%d %H:%M:%S")
        datos_consumos = traeSesiones(fecha_max_mintic,fecha_tope_mintic)
else:
    pass

In [46]:
datos_det_conex = trae_conexiones(fecha_max_mintic,fecha_tope_mintic)

In [47]:
datos_det_conex

,fechahora,mac_usuario,dispositivo,lugar,documento,tipodoc,lugar_cod,fecha_control,sistema_operativo
0,2021-06-01 00:00:03,24:DA:9B:02:9E:B9,Smartphone,CGTO RICAUTE_SAN JOSE DE CUCUTA,88214747,CC,70813-ZZZY057,2021-06-01 00:00:00,Android
1,2021-06-01 00:00:16,20:39:56:FD:9B:33,Smartphone,"B. TORRES MAJAY, MAICAO",1134171073,TI,37682-ZGYO918,2021-06-01 00:00:00,Android
2,2021-06-01 00:01:01,E6:A6:B0:D7:C4:B6,Smartphone,CALLE 6A 13 - 95_BARANOA,1042994172,TI,22483-ZGYO057,2021-06-01 00:00:00,Android
3,2021-06-01 00:01:06,E6:A6:B0:D7:C4:B6,Smartphone,CALLE 6A 13 - 95_BARANOA,1042994172,TI,22483-ZGYO057,2021-06-01 00:00:00,Android
4,2021-06-01 00:01:17,04:D3:95:A8:69:89,Smartphone,SAN LAZARO_ANTIOQUIA,1038930567,TI,20013-ZGY0178,2021-06-01 00:00:00,Android
5,2021-06-01 00:01:22,04:D3:95:A8:69:89,Smartphone,SAN LAZARO_ANTIOQUIA,1038930567,TI,20013-ZGY0178,2021-06-01 00:00:00,Android
6,2021-06-01 00:01:55,7C:A1:77:F9:BF:6A,Smartphone,COROZALITO_CHIMA,11077480,CC,31425-ZGYO307,2021-06-01 00:00:00,Android
7,2021-06-01 00:02:29,B0:68:E6:AD:D5:E9,PC,SHIRURI_MANAURE,40925748,CC,73868-ZGYO254,2021-06-01 00:00:00,Windows
8,2021-06-01 00:02:29,94:27:90:53:79:58,Smartphone,"POTRERITO, FONSECA",1120748322,CC,67709-ZGYO491,2021-06-01 00:00:00,Android
9,2021-06-01 00:30:14,40:5B:D8:4B:A4:DF,PC,VDA EL TOTUMO,1087992115,CC,47779-ZGYO194,2021-06-01 00:30:00,Windows


In [48]:
continuar = True
if datos_det_conex.empty:
    print("Dataframe vacio")
    print(fecha_max_mintic)
    print(fecha_tope_mintic)
    
    continuar = False
else:
    datos_det_conex['fecha'] = datos_det_conex['fecha_control'].str.split(" ", n = 1, expand = True)[0]
    datos_det_conex.drop_duplicates(subset=["fecha_control","lugar","lugar_cod","mac_usuario", "dispositivo","sistema_operativo",'tipodoc','documento'],inplace=True)

In [49]:
 datos_det_conex

,fechahora,mac_usuario,dispositivo,lugar,documento,tipodoc,lugar_cod,fecha_control,sistema_operativo,fecha
0,2021-06-01 00:00:03,24:DA:9B:02:9E:B9,Smartphone,CGTO RICAUTE_SAN JOSE DE CUCUTA,88214747,CC,70813-ZZZY057,2021-06-01 00:00:00,Android,2021-06-01
1,2021-06-01 00:00:16,20:39:56:FD:9B:33,Smartphone,"B. TORRES MAJAY, MAICAO",1134171073,TI,37682-ZGYO918,2021-06-01 00:00:00,Android,2021-06-01
2,2021-06-01 00:01:01,E6:A6:B0:D7:C4:B6,Smartphone,CALLE 6A 13 - 95_BARANOA,1042994172,TI,22483-ZGYO057,2021-06-01 00:00:00,Android,2021-06-01
4,2021-06-01 00:01:17,04:D3:95:A8:69:89,Smartphone,SAN LAZARO_ANTIOQUIA,1038930567,TI,20013-ZGY0178,2021-06-01 00:00:00,Android,2021-06-01
6,2021-06-01 00:01:55,7C:A1:77:F9:BF:6A,Smartphone,COROZALITO_CHIMA,11077480,CC,31425-ZGYO307,2021-06-01 00:00:00,Android,2021-06-01
7,2021-06-01 00:02:29,B0:68:E6:AD:D5:E9,PC,SHIRURI_MANAURE,40925748,CC,73868-ZGYO254,2021-06-01 00:00:00,Windows,2021-06-01
8,2021-06-01 00:02:29,94:27:90:53:79:58,Smartphone,"POTRERITO, FONSECA",1120748322,CC,67709-ZGYO491,2021-06-01 00:00:00,Android,2021-06-01
9,2021-06-01 00:30:14,40:5B:D8:4B:A4:DF,PC,VDA EL TOTUMO,1087992115,CC,47779-ZGYO194,2021-06-01 00:30:00,Windows,2021-06-01
10,2021-06-01 00:31:25,58:D9:C3:22:03:6A,Smartphone,"SAN JOSE DE CANALETE, PUERTO ESCONDIDO",1129804838,CC,32010-ZZZY346,2021-06-01 00:30:00,Android,2021-06-01
11,2021-06-01 00:31:34,D0:31:69:84:1B:53,Smartphone,PIEDRAS ABAJO,1065292603,CC,31505-ZZZY232,2021-06-01 00:30:00,Android,2021-06-01


In [50]:
if  continuar:
    
    datos_det_conex = datos_det_conex.rename(columns={'lugar_cod' : 'site_id'
                                                             ,'fechahora':'usuarios.fechaConexionUsuarios'
                                                             ,'dispositivo': 'usuarios.tipoDispositivoUsuarios'
                                                             , 'sistema_operativo': 'usuarios.sistemaOperativoUsuarios'})

In [51]:
datos_det_conex.columns

Index(['usuarios.fechaConexionUsuarios', 'mac_usuario',
       'usuarios.tipoDispositivoUsuarios', 'lugar', 'documento', 'tipodoc',
       'site_id', 'fecha_control', 'usuarios.sistemaOperativoUsuarios',
       'fecha'],
      dtype='object')

## Calculando total de registros

la lista use_these_keys se usa para referenciar cuales campos del dataframe irán al indice final. si los datos no se declaran en este, no se insertarán

Para cada sitio y fecha, se cuentan los documentos de usuario. Esto resulta en el total de usuarios registrados para esa fecha

In [52]:
use_these_keys = ['usuarios.fecha'
                  , 'usuarios.siteID'
                  , 'usuarios.usuariosRegistrados'
                  , 'usuarios.anyo'
                  , 'usuarios.mes'
                  , 'usuarios.dia'
                  , 'usuarios.nomCentroDigital'
                  , 'usuarios.codISO'
                  , 'usuarios.localidad'
                  , 'usuarios.nombreDepartamento'
                  , 'usuarios.sistemaEnergia'
                  , 'usuarios.nombreMunicipio'
                  , 'usuarios.idBeneficiario'
                  , 'usuarios.location'
                  , 'nombreDepartamento'
                    , 'nombreMunicipio'
                    , 'idBeneficiario'
                    , 'fecha'
                    , 'anyo'
                    , 'mes'
                    , 'dia'
                  , '@timestamp']

def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{'UsuRegistro-'+ str(document['usuarios.siteID']) + '-' + str(document['usuarios.fecha'])+ str(random.randrange(1000))}",
                "_source": filterKeys(document)
            }

In [53]:
def invalid_values(df, values=['']):
    return df.loc[df['usuarios.location'].isin(values)]

In [54]:
try:
    
    datos_registro['creado'] = datos_registro['creado'].str[0:10]
    datos_registro['tipo_mas_doc'] = datos_registro['tipodoc'] + datos_registro['documento']
    datos_registro= datos_registro.rename(columns={'creado' : 'usuarios.fecha'
                                                  ,'lugar_cod':'site_id'})

    datos_registro = datos_registro[["usuarios.fecha","site_id","tipo_mas_doc"]].groupby(["usuarios.fecha","site_id"]).agg(['count']).reset_index()
    datos_registro.columns = datos_registro.columns.droplevel(1)
    datos_registro= datos_registro.rename(columns={'tipo_mas_doc' : 'usuarios.usuariosRegistrados'})
    datos_registro["usuarios.anyo"] = datos_registro["usuarios.fecha"].str[0:4]
    datos_registro["usuarios.mes"] = datos_registro["usuarios.fecha"].str[5:7]
    datos_registro["usuarios.dia"] = datos_registro["usuarios.fecha"].str[8:10]
    datos_registro = pd.merge(datos_registro,  datos_semilla, on='site_id', how='inner')
    datos_registro.rename(columns={'site_id': 'usuarios.siteID'
                               }, inplace=True)
    
    datos_registro['nombreDepartamento'] = datos_registro['usuarios.nombreDepartamento']
    datos_registro['nombreMunicipio'] = datos_registro['usuarios.nombreMunicipio']
    datos_registro['idBeneficiario'] = datos_registro['usuarios.idBeneficiario']
    datos_registro['fecha'] = datos_registro['usuarios.fecha']
    datos_registro['anyo'] = datos_registro['usuarios.anyo']
    datos_registro['mes'] = datos_registro['usuarios.mes']
    datos_registro['dia'] = datos_registro['usuarios.dia']
    
    datos_registro['@timestamp'] = now.isoformat()
    
    salida = helpers.bulk(es, doc_generator(datos_registro))
    print("Fecha: ", now,"- Datos Registro usuarios en indice principal:",salida[0])
except Exception as e:
    print("Fecha: ", now,"- Nada que insertar en indice principal",e)

Fecha:  2021-08-17 22:13:38.600373 - Datos Registro usuarios en indice principal: 302


* se cruza por mac_usuario
* Los merge usan left para evitar perdida de datos en cruce con cambium-devicedevices. Aquellos datos que no cruzan se les marca como no identificados. En condiciones ideales, no debería presentarse ausencia de información
* Solo se toma lo que cruza con INNER
* se cambia el fabricante [Local MAC]
* se cambian lo nulos

In [55]:
try:
    datos_dev_clients.drop_duplicates(inplace=True)
    datos_dev_clients = datos_dev_clients.rename(columns={'ap_mac' : 'usuarios.macRed','mac' : 'mac_usuario'})
    datos_dev_clients['manufacturer'] = datos_dev_clients['manufacturer'].replace("[Local MAC]", "No identificado")
    datos_dev_clients.fillna({'manufacturer':'No identificado'},inplace=True)
    datos_det_conex = pd.merge(datos_det_conex,  datos_dev_clients, on='mac_usuario',how='left')
    datos_det_conex = pd.merge(datos_det_conex,datos_dev[['usuarios.macRed','usuarios.apGroup']],on='usuarios.macRed', how='left')
    datos_det_conex.fillna({'usuarios.tipoDispositivoUsuarios':'No identificado'
                       ,'usuarios.sistemaOperativoUsuarios':'No identificado'
                       ,'manufacturer':'No identificado'
                       ,'radio.band':'No identificado'
                       },inplace=True)
except:
    pass

## Calculando totales por dispositivos

la lista use_these_keys se usa para referenciar cuales campos del dataframe irán al indice final. si los datos no se declaran en este, no se insertarán

In [56]:
use_these_keys = ['usuarios.siteID'
                  , 'usuarios.nomCentroDigital'
                  , 'usuarios.codISO'
                  , 'usuarios.localidad'
                  , 'usuarios.nombreDepartamento'
                  , 'usuarios.sistemaEnergia'
                  , 'usuarios.nombreMunicipio'
                  , 'usuarios.idBeneficiario'
                  , 'usuarios.location'
                  , 'usuarios.sistemaOperativoUsuarios'
                  , 'usuarios.tipoDispositivoUsuarios'
                  , 'usuarios.marcaTerminal'
                  , 'usuarios.tecnologiaTerminal'
                  , 'usuarios.totales.dispositivos'
                  , 'usuarios.fechaControl'
                  , 'usuarios.fecha'
                  , 'usuarios.anyo'
                  , 'usuarios.mes'
                  , 'usuarios.dia'
                  , 'usuarios.hora'
                  , 'usuarios.minuto'
                  , 'nombreDepartamento'
                  , 'nombreMunicipio'
                  , 'idBeneficiario'
                  , 'fecha'
                  , 'anyo'
                  , 'mes'
                  , 'dia'
                  , '@timestamp']

def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{ 'totalesDispositivos-' + str(document['usuarios.siteID']) + '-' + str(document['usuarios.fechaControl'])+ str(random.randrange(1000))}",
                "_source": filterKeys(document),
            }

Se agrupa por:
* Sistema operativo
* Tipo dispositivo
* Marca del terminal
* Tecnología terminal

Se cuentan las mac de usuario para cada site id y fecha control

In [57]:
def cambioVariable(df):
    df = df.rename(columns={'siteID':'usuarios.tablero11.siteID'
                        ,'usuarios.siteID': 'usuarios.tablero11.siteID'
                        ,'usuarios.tablero11siteID':'usuarios.tablero11.siteID'
                        ,'fechaControl':'usuarios.tablero11.fechaControl'
                        ,'usuarios.fechaControl':'usuarios.tablero11.fechaControl'
                        ,'usuarios.tablero11fechaControl': 'usuarios.tablero11.fechaControl'
                        ,'nomCentroDigital':'usuarios.tablero11.nomCentroDigital'
                        ,'usuarios.nomCentroDigital':'usuarios.tablero11.nomCentroDigital'
                        ,'usuarios.tablero11nomCentroDigital':'usuarios.tablero11.nomCentroDigital'
                        ,'localidad':'usuarios.tablero11.localidad'
                        ,'usuarios.localidad':'usuarios.tablero11.localidad'
                        ,'usuarios.tablero11localidad':'usuarios.tablero11.localidad'
                        ,'nombreDepartamento':'usuarios.tablero11.nombreDepartamento'
                        ,'usuarios.nombreDepartamento':'usuarios.tablero11.nombreDepartamento'
                        ,'usuarios.tablero11nombreDepartamento':'usuarios.tablero11.nombreDepartamento'
                        ,'nombreMunicipio':'usuarios.tablero11.nombreMunicipio'
                        ,'usuarios.nombreMunicipio':'usuarios.tablero11.nombreMunicipio'
                        ,'usuarios.tablero11nombreMunicipio':'usuarios.tablero11.nombreMunicipio'
                        ,'idBeneficiario':'usuarios.tablero11.idBeneficiario'
                        ,'usuarios.idBeneficiario':'usuarios.tablero11.idBeneficiario'
                        ,'usuarios.tablero11idBeneficiario':'usuarios.tablero11.idBeneficiario'
                        ,'apGroup':'usuarios.tablero11.apGroup'
                        ,'usuarios.apGroup':'usuarios.tablero11.apGroup'
                        ,'usuarios.tablero11apGroup':'usuarios.tablero11.apGroup'
                        ,'location':'usuarios.tablero11.location'
                        ,'usuarios.location':'usuarios.tablero11.location'
                        ,'usuarios.tablero11location':'usuarios.tablero11.location'
                        ,'fecha':'usuarios.tablero11.fecha'
                        ,'usuarios.fecha':'usuarios.tablero11.fecha'
                        ,'usuarios.tablero11fecha':'usuarios.tablero11.fecha'
                        ,'anyo':'usuarios.tablero11.anyo'
                        ,'usuarios.anyo':'usuarios.tablero11.anyo'
                        ,'usuarios.tablero11anyo':'usuarios.tablero11.anyo'
                        ,'mes':'usuarios.tablero11.mes'
                        ,'usuarios.mes':'usuarios.tablero11.mes'
                        ,'usuarios.tablero11mes':'usuarios.tablero11.mes'
                        ,'macRed':'usuarios.tablero11.macRed'
                        ,'usuarios.macRed':'usuarios.tablero11.macRed'
                        ,'usuarios.tablero11macRed':'usuarios.tablero11.macRed'
                        ,'usuariosConectados':'usuarios.tablero11.usuariosConectados'
                        ,'usuarios.usuariosConectados':'usuarios.tablero11.usuariosConectados'
                        ,'usuarios.tablero11usuariosConectados':'usuarios.tablero11.usuariosConectados'
                        ,'sesiones_Usuarios':'usuarios.tablero11.sesiones_Usuarios'
                        ,'usuarios.sesiones_Usuarios':'usuarios.tablero11.sesiones_Usuarios'
                        ,'usuarios.tablero11sesiones_Usuarios':'usuarios.tablero11.sesiones_Usuarios'
                        ,'consumoUsuarios':'usuarios.tablero11.consumoUsuarios'
                        ,'usuarios.consumoUsuarios':'usuarios.tablero11.consumoUsuarios'
                        ,'usuarios.tablero11consumoUsuarios':'usuarios.tablero11.consumoUsuarios'
                        ,'consumoUsuariosDescarga_aux':'usuarios.tablero11.consumoUsuariosDescarga_aux'
                        ,'usuarios.consumoUsuariosDescarga_aux':'usuarios.tablero11.consumoUsuariosDescarga_aux'
                        ,'usuarios.tablero11consumoUsuariosDescarga_aux':'usuarios.tablero11.consumoUsuariosDescarga_aux'
                        ,'consumoUsuariosCarga_aux':'usuarios.tablero11.consumoUsuariosCarga_aux'
                        ,'usuarios.consumoUsuariosCarga_aux':'usuarios.tablero11.consumoUsuariosCarga_aux'
                        ,'usuarios.tablero11consumoUsuariosCarga_aux':'usuarios.tablero11.consumoUsuariosCarga_aux'
                        ,'consumoUsuariosDescarga':'usuarios.tablero11.consumoUsuariosDescarga'
                        ,'usuarios.consumoUsuariosDescarga':'usuarios.tablero11.consumoUsuariosDescarga'
                        ,'usuarios.tablero11consumoUsuariosDescarga':'usuarios.tablero11.consumoUsuariosDescarga'
                        ,'consumoUsuariosCarga': 'usuarios.tablero11.consumoUsuariosCarga'
                        ,'usuarios.tablero11consumoUsuariosCarga': 'usuarios.tablero11.consumoUsuariosCarga'
                       }
              )
    
    df_vacio = pd.DataFrame(index=df.index)
    
    col_select= ['usuarios.tablero11.siteID'
                 ,'usuarios.tablero11.nomCentroDigital'
                 , 'usuarios.tablero11.localidad'
                 , 'usuarios.tablero11.nombreDepartamento'
                 , 'usuarios.tablero11.nombreMunicipio'
                 , 'usuarios.tablero11.idBeneficiario'
                 , 'usuarios.tablero11.location'
                 , 'usuarios.tablero11.sesiones_Usuarios'
                 , 'usuarios.tablero11.usuariosConectados'
                 , 'usuarios.tablero11.consumoUsuarios'
                 , 'usuarios.tablero11.consumoUsuariosDescarga_aux'
                 , 'usuarios.tablero11.consumoUsuariosCarga_aux'
                 , 'usuarios.tablero11.consumoUsuariosDescarga'
                 , 'usuarios.tablero11.consumoUsuariosCarga'
                 , 'usuarios.tablero11.apGroup'
                 , 'usuarios.tablero11.fechaControl'
                 , 'usuarios.tablero11.fecha'
                 , 'usuarios.tablero11.anyo'
                 , 'usuarios.tablero11.mes'
                 , 'usuarios.tablero11.macRed'
                 , '@timestamp']
    
    for c in col_select:
        try:
            df_vacio=pd.concat([df_vacio,df[c]],axis=1)
        except:
            pass
    
#     print(df.columns)
    
    df_vacio=df_vacio.iloc[:,~df_vacio.columns.duplicated()]

    return df_vacio
    
#     df= df[['usuarios.tablero11.siteID','usuarios.tablero11.fechaControl','usuarios.tablero11.nomCentroDigital',\
#            'usuarios.tablero11.localidad','usuarios.tablero11.nombreDepartamento','usuarios.tablero11.nombreMunicipio', \
#           'usuarios.tablero11idBeneficiario','usuarios.tablero11.apGroup','usuarios.tablero11.location', \
#           'usuarios.tablero11.fecha','usuarios.tablero11.anyo','usuarios.tablero11.mes','usuarios.tablero11.macRed' \
#           'usuarios.tablero11.usuariosConectados','usuarios.tablero11.sesiones_Usuarios','usuarios.tablero11.consumoUsuarios' \
#           'usuarios.tablero11.consumoUsuariosDescarga_aux','usuarios.tablero11.consumoUsuariosCarga_aux',\
#            'usuarios.tablero11.consumoUsuariosDescarga','usuarios.tablero11.consumoUsuariosCarga','@timestamp']]
    
#     return df

In [58]:
try:
    totalesDispositivos = datos_det_conex[["fecha_control","site_id"
                                 ,"usuarios.sistemaOperativoUsuarios"
                                 ,'usuarios.tipoDispositivoUsuarios'
                                 ,'manufacturer'
                                 ,'radio.band'
                                 ,"mac_usuario"]].groupby(["fecha_control","site_id"
                                                     ,"usuarios.sistemaOperativoUsuarios"
                                                     ,'usuarios.tipoDispositivoUsuarios'
                                                     ,'manufacturer'
                                                     ,'radio.band']).agg(['count']).reset_index()
    ##si se quiere contar ocurrencias unicas se debe usar en lugar del agg: ['mac_usuario'].nunique().reset_index()
    totalesDispositivos.columns = totalesDispositivos.columns.droplevel(1)
    totalesDispositivos= totalesDispositivos.rename(columns={'mac_usuario' : 'usuarios.totales.dispositivos'
                                                            ,'fecha_control' : 'usuarios.fechaControl'
                                                            ,'manufacturer' : 'usuarios.marcaTerminal'
                                                            ,'radio.band' : 'usuarios.tecnologiaTerminal'
                                                            })
    totalesDispositivos = pd.merge(totalesDispositivos, datos_semilla, on='site_id',how='inner')
    totalesDispositivos["usuarios.fecha"] = totalesDispositivos["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    totalesDispositivos["usuarios.anyo"] = totalesDispositivos["usuarios.fecha"].str[0:4]
    totalesDispositivos["usuarios.mes"] = totalesDispositivos["usuarios.fecha"].str[5:7]
    totalesDispositivos["usuarios.dia"] = totalesDispositivos["usuarios.fecha"].str[8:10]
    totalesDispositivos["usuarios.hora"] = totalesDispositivos["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    totalesDispositivos["usuarios.minuto"] = totalesDispositivos["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    totalesDispositivos= totalesDispositivos.rename(columns={'site_id' : 'usuarios.siteID'})
    totalesDispositivos['nombreDepartamento'] = totalesDispositivos['usuarios.nombreDepartamento']
    totalesDispositivos['nombreMunicipio'] = totalesDispositivos['usuarios.nombreMunicipio']
    totalesDispositivos['idBeneficiario'] = totalesDispositivos['usuarios.idBeneficiario']
    totalesDispositivos['fecha'] = totalesDispositivos['usuarios.fecha']
    totalesDispositivos['anyo'] = totalesDispositivos['usuarios.anyo']
    totalesDispositivos['mes'] = totalesDispositivos['usuarios.mes']
    totalesDispositivos['dia'] = totalesDispositivos['usuarios.dia']
    totalesDispositivos['@timestamp'] = now.isoformat()
    salida = helpers.bulk(es, doc_generator(totalesDispositivos))
    print("Fecha: ", now,"- Totales por Dispositivos insertados en indice principal:",salida[0])
except:
    print("Fecha: ", now,"- No se insertaron totales por dispositivos en indice principal")

Fecha:  2021-08-17 22:13:38.600373 - Totales por Dispositivos insertados en indice principal: 9


# Insertando usuarios conectados al indice principal

* Se calcula cantidad de sesiones por sitio con detalle conexiones, pero contando la ocurrencia unica de documento
* Se calcula la cantidad de conexiones. Se agrupa por el campo usuarios.macRed(el cual corresponde al AP) y fecha_control. Se cuenta las ocurrencias de mac_usuario. Luego al cruzar con flujo principal, si el dato es nulo para ese momento, se debe colocar en 0.

In [59]:
fecha_max_mintic = fecha_ejecucion

fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_consumos = traeSesiones(fecha_max_mintic,fecha_tope_mintic)

try:

    if datos_consumos is None or datos_consumos.empty:
        while (datos_consumos is None or datos_consumos.empty) and ((datetime.strptime(fecha_max_mintic[0:50], '%Y-%m-%d %H:%M:%S"').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
            fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
            fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
            datos_consumos = traeSesiones(fecha_max_mintic,fecha_tope_mintic)
    else:
        pass
except:
    pass

In [60]:
datos_logins = datos_det_conex[['fecha_control', 'site_id', 'documento','usuarios.macRed']].groupby(["fecha_control","site_id","usuarios.macRed"])['documento'].nunique().reset_index()
datos_logins= datos_logins.rename(columns={'documento' : 'usuarios.sesiones_Usuarios'})

In [1]:
 try:
    datos_logins = datos_det_conex[['fecha_control', 'site_id', 'documento']].groupby(["fecha_control","site_id"])['documento'].nunique().reset_index()
    datos_logins= datos_logins.rename(columns={'documento' : 'usuarios.sesiones_Usuarios'})

    ###################################################################################################


    #Se trae usuarios conectados desde cambium devices clients 
    usuariosConectados = usuarios_conectados_cambium[["fecha_control","site_id","mac_usuario"]].groupby(["fecha_control","site_id"]).agg(['count']).reset_index()
    usuariosConectados.columns = usuariosConectados.columns.droplevel(1)
    usuariosConectados= usuariosConectados.rename(columns={'mac_usuario' : 'usuarios.usuariosConectados'})


    ###################################################################################################

    usuariosConectados = pd.merge(usuariosConectados,datos_logins,  how='outer')
    usuariosConectados.fillna({'usuarios.usuariosConectados': 0
                               ,'usuarios.sesiones_Usuarios' : 0 },inplace=True)
    usuariosConectados['usuarios.usuariosConectados'] = usuariosConectados['usuarios.usuariosConectados'].astype(int)
    usuariosConectados['usuarios.sesiones_Usuarios'] = usuariosConectados['usuarios.sesiones_Usuarios'].astype(int)
    usuariosConectados = pd.merge(datos_semilla,  usuariosConectados, on=['site_id'], how='inner')
    usuariosConectados = usuariosConectados.rename(columns={'fecha_control':'usuarios.fechaControl'
                                                           ,'site_id' : 'usuarios.siteID'})


    try:
        usuariosConectados["usuarios.fecha"] = usuariosConectados["usuarios.fechaControl"].str[0:10]
    except:
        usuariosConectados["usuarios.fecha"] = ""

    try:
        usuariosConectados["usuarios.anyo"] = usuariosConectados["usuarios.fecha"].str[0:4]
    except:
        usuariosConectados["usuarios.anyo"] = ""

    try:
        usuariosConectados["usuarios.mes"] = usuariosConectados["usuarios.fecha"].str[5:7]
    except:
        usuariosConectados["usuarios.mes"] = ""

    try:
        usuariosConectados["usuarios.dia"] = usuariosConectados["usuarios.fecha"].str[8:10]
    except:
        usuariosConectados["usuarios.dia"] = ""



    usuariosConectados['nombreDepartamento'] = usuariosConectados['usuarios.nombreDepartamento']
    usuariosConectados['nombreMunicipio'] = usuariosConectados['usuarios.nombreMunicipio']
    usuariosConectados['idBeneficiario'] = usuariosConectados['usuarios.idBeneficiario']
    usuariosConectados['fecha'] = usuariosConectados['usuarios.fecha']
    usuariosConectados['anyo'] = usuariosConectados['usuarios.anyo']
    usuariosConectados['mes'] = usuariosConectados['usuarios.mes']
    usuariosConectados['dia'] = usuariosConectados['usuarios.dia']
    usuariosConectados['@timestamp'] = now.isoformat()

except:
    print("Fecha: ", now,"- Ningun usuario conectado para insertar en indice principal:")

NameError: name 'now' is not defined

la lista use_these_keys se usa para referenciar cuales campos del dataframe irán al indice final. si los datos no se declaran en este, no se insertarán

In [62]:
try:
    use_these_keys = ['usuarios.nomCentroDigital'
                  , 'usuarios.codISO'
                  , 'usuarios.idBeneficiario'    
                  , 'usuarios.localidad'
                  , 'usuarios.siteID'
                  , 'usuarios.nombreDepartamento'
                  , 'usuarios.sistemaEnergia'
                  , 'usuarios.nombreMunicipio'
                  , 'usuarios.location'
                  , 'usuarios.fechaControl'
                  #, 'usuarios.macRed'
                  #, 'usuarios.apGroup'    
                  , 'usuarios.usuariosConectados'
                  , 'usuarios.sesiones_Usuarios'
                  , 'usuarios.fecha'
                  , 'usuarios.anyo'
                  , 'usuarios.mes'
                  , 'usuarios.dia'
                  #, 'usuarios.hora'
                  #, 'usuarios.minuto'
                  , 'nombreDepartamento'
                  , 'nombreMunicipio'
                  , 'idBeneficiario'
                  , 'fecha'
                  , 'anyo'
                  , 'mes'
                  , 'dia'
                  , '@timestamp']

    usuariosConectados['@timestamp'] = now.isoformat()
    def doc_generator(df):
        df_iter = df.iterrows()
        for index, document in df_iter:
            yield {
                    "_index": indice, 
                    "_id": f"{ str(document['usuarios.siteID']) + '-' + str(document['usuarios.fechaControl']) + '-' +str(random.randrange(10000000))}",
                    "_source": filterKeys(document),
                }
    salida = helpers.bulk(es, doc_generator(usuariosConectados))
    print("Fecha: ", now,"- Usuarios conectados insertados en indice principal:",salida[0])
except:
    print("Fecha: ", now,"- Ningun usuario conectado para insertar en indice principal:")

Fecha:  2021-08-17 22:13:38.600373 - Usuarios conectados insertados en indice principal: 20865


### Se lee el indice all-cambium-device-client

* En este indice se guarda el detalle de los radio por fecha
* Detalle conexiones cruza con device clients. Con estos se calculan los totales por marca

In [63]:
total_docs = 100000
response = es.search(
    index= 'all-'+parametros.cambium_d_c_index,
    body={
        "_source": ['mac', 'ap_mac', 'radio.band', 'radio.rx_bytes', 'radio.tx_bytes','fecha_control']
          , "query": {
              "range": {
                "fecha_control": {
                  "gte": fecha_max_mintic,
                  "lt": fecha_tope_mintic
                  #"gte": "2021-05-26 15:00:00",
                  #"lt": "2021-05-26 15:10:00"  
                }
              }
          }
    },
    size=total_docs
)
elastic_docs = response["hits"]["hits"]
#fields = {}
#for num, doc in enumerate(elastic_docs):
#    source_data = doc["_source"]
#   for key, val in source_data.items():
#        try:
#            fields[key] = np.append(fields[key], val)
#        except KeyError:
#            fields[key] = np.array([val])

#datos_radio = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))

datos_radio = pd.DataFrame([x["_source"] for x in elastic_docs])

datos_performance = datos_radio

In [64]:
if datos_performance is None or datos_performance.empty:
    while (datos_performance is None or datos_performance.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
        datos_performance = traeRadio(fecha_max_mintic,fecha_tope_mintic)
else:
    pass

## Asociando datos de Speed test

Se tiene una lectura diara de velocidad para cada centro. Por tanto se debe cruzar con el fjulo principal, haciendo uso solo del año, mes día, sin incluir la hora.

In [65]:
def traeVelocidad(fecha_max_mintic,fecha_tope_mintic):
    total_docs = 10000
    response = es.search(
        index= parametros.speed_index+'*',
        body={
                "_source": ["beneficiary_code","locationid", "result_start_date"
                            , "result_download_mbps", "result_upload_mbps"],
                "query": {
                    "range": {
                        "result_start_date": {
                            "gte": fecha_max_mintic.split(' ')[0]+'T00:00:00',
                            "lt": fecha_tope_mintic.split(' ')[0]+'T23:59:59'
                        }
                    }
                }

        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))


Se genera fecha en yyyy-mm-dd, y cada campo por separado
La hora y minuto se toma aparte
Valores que se convierten a cero si son nulos

trafico.anchoBandaDescarga
trafico.anchoBandaCarga

In [66]:
fecha_max_mintic = fecha_ejecucion
fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_speed = traeVelocidad(fecha_max_mintic,fecha_tope_mintic)

if datos_speed is None or datos_speed.empty:
    while (datos_speed is None or datos_speed.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)).strftime("%Y-%m-%d %H:%M:%S")
        datos_speed = traeVelocidad(fecha_max_mintic,fecha_tope_mintic)
else:
    pass

## 7. Speed test a indice

In [67]:
use_these_keys = ['trafico.nomCentroDigital', 
                  'trafico.localidad',
                  'trafico.siteID',
                  'trafico.nombreDepartamento', 
                  'trafico.codISO', 
                  'trafico.sistemaEnergia',
                  'trafico.nombreMunicipio', 
                  'trafico.idBeneficiario',
                  'trafico.location', 
                  'trafico.anchoBandaDescarga',
                  'trafico.anchoBandaCarga',
                  'trafico.totales.fecha',
                  'trafico.totales.anyo',
                  'trafico.totales.mes',
                  'trafico.totales.dia',
                  'nombreDepartamento',
                    'nombreMunicipio',
                    'idBeneficiario',
                    'fecha',
                    'anyo',
                    'mes',
                    'dia',
                  '@timestamp']

#def doc_generator(df):
#    df_iter = df.iterrows()
#    for index, document in df_iter:
#        yield {
#                "_index": indice, 
#                "_id": f"{ 'Velocidad-' + document['trafico.siteID'] + '-' + document['trafico.totales.fecha']}",
#                "_source": filterKeys(document),
#            }

## leyendo indice ohmyfi valoraciones

Se toman en cuenta todas las valoraciones realizadas dentro del rango de fecha fecha. Campos leidos:

lugar_cod que es la llave para cruzar con site_id.
datos asociados a la valoración: pregunta, respuesta, fechahora (cuando se registró la valoración)

In [68]:
def trae_valoraciones(fecha_ini,fecha_fin):
    total_docs = 1000000
    response = es.search(
        index= parametros.ohmyfi_val_index,
        body={
                  "_source": ["lugar_cod", "respuesta","fechahora","@timestamp"]
                , "query": {
                      "range": {
                            "fechahora": {
                            "gte": fecha_ini,
                            "lt":  fecha_fin
                            }
                        }
                  }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    #fields = {}
    #for num, doc in enumerate(elastic_docs):
    #    source_data = doc["_source"]
    #    for key, val in source_data.items():
    #        try:
    #            fields[key] = np.append(fields[key], val)
    #        except KeyError:
    #            fields[key] = np.array([val])

    #return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) 
    return pd.DataFrame([x["_source"] for x in elastic_docs])

In [70]:
#trae_valoraciones

## Se realiza la consulta de datos

Se calcula rango en base a la fecha de control. Para este caso es de un día.
Se ejecuta la función de consulta con el rango de fechas.
Si no retorna datos se incrementa el rango y se ejecuta nuevamente. Este proceso se repite hasta conseguir datos o hasta que el rango de ejecución alcance la fecha y hora actual.

In [71]:
fecha_max_mintic = fecha_ejecucion
fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_valoraciones =  trae_valoraciones(fecha_max_mintic,fecha_tope_mintic)

if datos_valoraciones is None or datos_valoraciones.empty:
    while (datos_valoraciones is None or datos_valoraciones.empty) and ((datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
        datos_valoraciones = trae_valoraciones(fecha_max_mintic,fecha_tope_mintic)
else:
    pass

In [72]:
datos_valoraciones

,fechahora,@timestamp,lugar_cod,respuesta
0,2021-06-01 00:17:51,2021-06-08T13:50:03.011597,47824-ZZZY166,No
1,2021-06-01 00:22:39,2021-06-08T13:50:03.011597,47824-ZZZY166,No
2,2021-06-01 00:01:21,2021-06-08T13:50:03.011597,20013-ZGY0178,No
3,2021-06-01 00:53:46,2021-06-08T13:50:03.011597,20013-ZGY0178,No
4,2021-06-01 00:46:31,2021-06-08T13:50:03.011597,31565-ZGYO325,Si
...,...,...,...,...
49812,2021-06-01 23:26:10,2021-06-08T17:01:37.107373,30374-ZGYO439,Excelente
49813,2021-06-01 23:26:12,2021-06-08T17:01:37.107373,30374-ZGYO439,Excelente
49814,2021-06-01 23:16:31,2021-06-08T17:01:37.107373,31993-ZGYO266,Si
49815,2021-06-01 23:16:44,2021-06-08T17:01:37.107373,31993-ZGYO266,Excelente


Se descartan las respuestas Si y No de las valoraciones, las cuales corresponden a la pregunta: "Te gustaría calificar tu última conexión en". De esta forma solo se dejan las respuestas asociadas a la percepción de calidad por parte del usuario.
Se estandariza lugar_cod a site_id
Se estadariza fecha para agrupar (solo se toma yyyy-mm-dd)
Se Calcula nivel de valoracion por pregunta y total de valoraciones
Para cada centro de conexión, pregunta, se contabilizan las valoraciones. El calculo es diario

# Uso servicio de internet insertados en indice principal

la lista use_these_keys se usa para referenciar cuales campos del dataframe irán al indice final. si los datos no se declaran en este, no se insertarán

In [73]:
use_these_keys = ['usuarios.macRed'
                  , 'usuarios.usoServicioInternetSitio'
                  , 'usuarios.detallesTecnologiasTerminales'
                  , 'usuarios.nomCentroDigital'
                  , 'usuarios.codISO'
                  , 'usuarios.idBeneficiario'
                  , 'usuarios.localidad'
                  , 'usuarios.siteID'
                  , 'usuarios.nombreDepartamento'
                  , 'usuarios.sistemaEnergia'
                  , 'usuarios.nombreMunicipio'
                  , 'usuarios.location'
                  , 'usuarios.fechaControl'
                  , 'usuarios.fecha'
                  , 'usuarios.anyo'
                  , 'usuarios.mes'
                  , 'usuarios.dia'
                  , 'usuarios.hora'
                  , 'usuarios.minuto'
                  , 'usuarios.apGroup'
                  , 'nombreDepartamento'
                    , 'nombreMunicipio'
                    , 'idBeneficiario'
                    , 'fecha'
                    , 'anyo'
                    , 'mes'
                    , 'dia'
                  , '@timestamp']

def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{ 'UsoServicio-' + str(document['usuarios.macRed']) + '-' + str(document['usuarios.detallesTecnologiasTerminales']) + '-' + str(document['usuarios.fechaControl'])+ str(random.randrange(1000))}",
                "_source": filterKeys(document),
            }

* se toma el maximo radio para cada mac de dispositivo final
* se suma agrupando por ap_mac, banda y fecha
* El resultado es un total para cada banda, fecha y ap_mac

In [74]:
try:    
    datos_radio['fecha_control'] = datos_radio['fecha_control'].str[0:15]+'0:00'
    datos_radio = datos_radio.groupby(['ap_mac', 'fecha_control', 'mac', 'radio.band']).agg(['max']).reset_index()
    datos_radio.columns = datos_radio.columns.droplevel(1)
    datos_radio = datos_radio[['ap_mac', 'fecha_control', 'radio.band', 'radio.rx_bytes',
           'radio.tx_bytes']].groupby(['ap_mac', 'fecha_control', 'radio.band']).agg(['sum']).reset_index()
    datos_radio.columns = datos_radio.columns.droplevel(1)
    datos_radio = datos_radio.rename(columns={'ap_mac' : 'usuarios.macRed'})
    usoServicioInternetSitio = pd.merge(datos_radio,datos_dev[['usuarios.apGroup', 'site_id', 'usuarios.macRed']], on ='usuarios.macRed', how='inner')
    usoServicioInternetSitio.fillna({'radio.rx_bytes':0,'radio.tx_bytes':0},inplace=True)
    usoServicioInternetSitio.fillna({'usuarios.macRed':'','radio.band':'No identificado','usuarios.apGroup':'No identificado'},inplace=True)
    usoServicioInternetSitio['usuarios.usoServicioInternetSitio'] = usoServicioInternetSitio['radio.rx_bytes'].astype(float) + usoServicioInternetSitio['radio.tx_bytes'].astype(float)
    usoServicioInternetSitio = pd.merge(usoServicioInternetSitio,datos_semilla, on ='site_id', how='inner')
    usoServicioInternetSitio= usoServicioInternetSitio.rename(columns={'site_id' : 'usuarios.siteID'
                                                                    ,'radio.band' : 'usuarios.detallesTecnologiasTerminales'
                                                                    , 'fecha_control' : 'usuarios.fechaControl'})
    usoServicioInternetSitio['usuarios.usoServicioInternetSitio'] = round((usoServicioInternetSitio['usuarios.usoServicioInternetSitio']/float(1<<30)),6) 
    usoServicioInternetSitio["usuarios.fecha"] = usoServicioInternetSitio["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    usoServicioInternetSitio["usuarios.anyo"] = usoServicioInternetSitio["usuarios.fecha"].str[0:4]
    usoServicioInternetSitio["usuarios.mes"] = usoServicioInternetSitio["usuarios.fecha"].str[5:7]
    usoServicioInternetSitio["usuarios.dia"] = usoServicioInternetSitio["usuarios.fecha"].str[8:10]
    usoServicioInternetSitio["usuarios.hora"] = usoServicioInternetSitio["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    usoServicioInternetSitio["usuarios.minuto"] = usoServicioInternetSitio["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    
    usoServicioInternetSitio['nombreDepartamento'] = usoServicioInternetSitio['usuarios.nombreDepartamento']
    usoServicioInternetSitio['nombreMunicipio'] = usoServicioInternetSitio['usuarios.nombreMunicipio']
    usoServicioInternetSitio['idBeneficiario'] = usoServicioInternetSitio['usuarios.idBeneficiario']
    usoServicioInternetSitio['fecha'] = usoServicioInternetSitio['usuarios.fecha']
    usoServicioInternetSitio['anyo'] = usoServicioInternetSitio['usuarios.anyo']
    usoServicioInternetSitio['mes'] = usoServicioInternetSitio['usuarios.mes']
    usoServicioInternetSitio['dia'] = usoServicioInternetSitio['usuarios.dia']
    
    usoServicioInternetSitio['@timestamp'] = now.isoformat()
    salida = helpers.bulk(es, doc_generator(usoServicioInternetSitio))
    print("Fecha: ", now,"- Uso servicio y tecnología terminal insertados en indice principal:",salida[0])
except:
    print("Fecha: ", now,"- Ningun uservicio/tecnología terminal conectado para insertar en indice principal")

Fecha:  2021-08-17 22:13:38.600373 - Uso servicio y tecnología terminal insertados en indice principal: 9874


Se renombra fecha_control para cruzar

## Totalizar la cantidad de dispositivos por usuario

* Para contabilizar los dispositivos por usuario, se debe contar la cantidad de ocurrencias de mac_usuario dentro de detalle de conexiones, agrupando por tipo documento y documento. 
* Aún cuando la mayoria de las fuentes referencia a mac_usuario, no se debe confundir a la hora de agrupar con los datos reales de usuario que en este caso serían el documento de identidad como campo de identificación

In [75]:
try:
    conteoDispositivos = datos_det_conex[['fecha_control','site_id', 'documento', 'tipodoc','fecha']].groupby(['fecha_control','site_id', 'documento', 'tipodoc']).agg(['count']).reset_index()
    conteoDispositivos.columns = conteoDispositivos.columns.droplevel(1)
    conteoDispositivos= conteoDispositivos.rename(columns={'fecha' : 'conteo'})
    conteoDispositivos = conteoDispositivos[['fecha_control','site_id','conteo']].groupby(['fecha_control','site_id']).agg(['sum']).reset_index()
    conteoDispositivos.columns = conteoDispositivos.columns.droplevel(1)
    conteoDispositivos= conteoDispositivos.rename(columns={'conteo' : 'usuarios.conteoDispositivos'})
    conteoDispositivos = pd.merge(conteoDispositivos, datos_semilla, on='site_id', how='inner')
    conteoDispositivos = conteoDispositivos.rename(columns={'fecha_control' : 'usuarios.fechaControl'})
    conteoDispositivos["usuarios.fecha"] = conteoDispositivos["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    conteoDispositivos["usuarios.anyo"] = conteoDispositivos["usuarios.fecha"].str[0:4]
    conteoDispositivos["usuarios.mes"] = conteoDispositivos["usuarios.fecha"].str[5:7]
    conteoDispositivos["usuarios.dia"] = conteoDispositivos["usuarios.fecha"].str[8:10]
    conteoDispositivos["usuarios.hora"] = conteoDispositivos["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    conteoDispositivos["usuarios.minuto"] = conteoDispositivos["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    conteoDispositivos= conteoDispositivos.rename(columns={'site_id' : 'usuarios.siteID'})
    
    conteoDispositivos['nombreDepartamento'] = conteoDispositivos['usuarios.nombreDepartamento']
    conteoDispositivos['nombreMunicipio'] = conteoDispositivos['usuarios.nombreMunicipio']
    conteoDispositivos['idBeneficiario'] = conteoDispositivos['usuarios.idBeneficiario']
    conteoDispositivos['fecha'] = conteoDispositivos['usuarios.fecha']
    conteoDispositivos['anyo'] = conteoDispositivos['usuarios.anyo']
    conteoDispositivos['mes'] = conteoDispositivos['usuarios.mes']
    conteoDispositivos['dia'] = conteoDispositivos['usuarios.dia']
except:
    pass

# 4. Insertando promedio dispositivos en indice principal

la lista use_these_keys se usa para referenciar cuales campos del dataframe irán al indice final. si los datos no se declaran en este, no se insertarán

In [76]:
try:
    use_these_keys = ['usuarios.nomCentroDigital'
                  , 'usuarios.codISO'
                  , 'usuarios.idBeneficiario'    
                  , 'usuarios.localidad'
                  , 'usuarios.siteID'
                  , 'usuarios.nombreDepartamento'
                  , 'usuarios.sistemaEnergia'
                  , 'usuarios.nombreMunicipio'
                  , 'usuarios.location'
                  , 'usuarios.conteoDispositivos'
                  , 'usuarios.fechaControl'
                  , 'usuarios.fecha'
                  , 'usuarios.anyo'
                  , 'usuarios.mes'
                  , 'usuarios.dia'
                  , 'usuarios.hora'
                  , 'usuarios.minuto'
                    , 'nombreDepartamento'
                    , 'nombreMunicipio'
                    , 'idBeneficiario'
                    , 'fecha'
                    , 'anyo'
                    , 'mes'
                    , 'dia'
                  , '@timestamp']

    conteoDispositivos['@timestamp'] = now.isoformat()
    def doc_generator(df):
        df_iter = df.iterrows()
        for index, document in df_iter:
            yield {
                    "_index": indice, 
                    "_id": f"{ 'PromedioDispositivos-' + str(document['usuarios.siteID']) + '-' + str(document['usuarios.fechaControl'])+str(random.randrange(1000))}",
                    "_source": filterKeys(document),
                }
    salida = helpers.bulk(es, doc_generator(conteoDispositivos))
    print("Fecha: ", now,"- Promedios dispositivos Usuarios:",salida[0])
except:
    print("Fecha: ", now,"- Ningun Promedios dispositivos Usuarios para insertar en indice principal")

Fecha:  2021-08-17 22:13:38.600373 - Promedios dispositivos Usuarios: 9


# Marcando si usuario es nuevo o recurrente

* Se toma del dataframe datos_det_conex y se cuenta las veces que cada mac_usuario aparece
* Luego se suma con el historico de conexiones por usuario (indice intermedio ohmyfi-total-conexiones-historico)
* marca como Nuevo(1 conexión). 

De este proceso se obtiene el campo:
* usuarios.usuariosNuevos
* usuarios.totales.usuariosNuevos

In [77]:
try:
    datos_recurrencia = datos_det_conex[["fecha","tipodoc","documento"]].groupby(["tipodoc","documento"]).agg(['count']).reset_index()
    datos_recurrencia.columns = datos_recurrencia.columns.droplevel(1)
    datos_recurrencia= datos_recurrencia.rename(columns={'fecha' : 'usuarios_recurrencia_aux'})
    datos_recurrencia['tipodoc'] = datos_recurrencia['tipodoc'].replace('','No especificado')
    datos_recurrencia['documento'] = datos_recurrencia['documento'].replace('','No especificado')
except:
    pass

### Se lee historico

Se lee ohmyfi-total-conexiones-historico, el cual es un indice creado a partir de los datos crudos. Este guarda la cantidad de conexiones realizadas por ese usuario. Para que la consulta funcione correctamente se debe colocar el filtro que usuarios_recurrencia exista.

In [78]:
try:
    total_docs = 30000000
    response = es.search(
        index= parametros.ohmyfi_total_c_index,
        body={
              "_source": ['tipodoc', 'documento', 'usuarios_recurrencia'],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"usuarios_recurrencia"
                    }
                  }
                  ]
                }
              }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
#    fields = {}
#    for num, doc in enumerate(elastic_docs):
#       source_data = doc["_source"]
#       for key, val in source_data.items():
#            try:
#                fields[key] = np.append(fields[key], val)
#            except KeyError:
#               fields[key] = np.array([val])

    #ya_en_indice = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))
    
    ya_en_indice = pd.DataFrame([x["_source"] for x in elastic_docs])
    
except:
    ya_en_indice = pd.DataFrame(columns=['tipodoc', 'documento', 'usuarios_recurrencia'])

In [79]:
if ya_en_indice is None or ya_en_indice.empty:
    datos_recurrencia= datos_recurrencia.rename(columns={'usuarios_recurrencia_aux' : 'usuarios_recurrencia'})
else:
    datos_recurrencia = pd.merge(datos_recurrencia, ya_en_indice, on=['tipodoc', 'documento'],how='left')
    datos_recurrencia.fillna({'usuarios_recurrencia':0},inplace=True)
    datos_recurrencia['usuarios_recurrencia'] = datos_recurrencia['usuarios_recurrencia'] + datos_recurrencia['usuarios_recurrencia_aux']
    datos_recurrencia['usuarios_recurrencia'] = [int(x) for x in datos_recurrencia['usuarios_recurrencia']]

In [80]:
try:
    aux_recurrencia = pd.merge(datos_recurrencia,datos_det_conex, on=['tipodoc','documento'],how='inner' )
    aux_recurrencia = aux_recurrencia[(aux_recurrencia['usuarios_recurrencia']==1)]
    aux_recurrencia.loc[aux_recurrencia['usuarios_recurrencia']==1,'usuarios.usuariosNuevos']='Nuevo'
    #aux_recurrencia.loc[aux_recurrencia['usuarios_recurrencia']!=1,'usuarios.usuariosNuevos']='Recurrente'
    aux_recurrencia = aux_recurrencia.rename(columns={'usuarios_recurrencia' :'usuarios.sesiones_Usuarios'})
    aux_recurrencia = aux_recurrencia[["fecha_control","site_id","usuarios.usuariosNuevos",'documento']].groupby(["fecha_control","site_id","usuarios.usuariosNuevos"])['documento'].nunique().reset_index()
    aux_recurrencia= aux_recurrencia.rename(columns={'documento' : 'usuarios.totales.usuariosNuevos'})
except:
    pass

In [81]:
try:
    datos_recurrencia = pd.merge(datos_semilla,  aux_recurrencia, on='site_id', how='inner')
    datos_recurrencia = datos_recurrencia.rename(columns={'fecha_control' : 'usuarios.fechaControl'})
    datos_recurrencia["usuarios.fecha"] = datos_recurrencia["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    datos_recurrencia["usuarios.anyo"] = datos_recurrencia["usuarios.fecha"].str[0:4]
    datos_recurrencia["usuarios.mes"] = datos_recurrencia["usuarios.fecha"].str[5:7]
    datos_recurrencia["usuarios.dia"] = datos_recurrencia["usuarios.fecha"].str[8:10]
    datos_recurrencia["usuarios.hora"] = datos_recurrencia["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    datos_recurrencia["usuarios.minuto"] = datos_recurrencia["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    datos_recurrencia= datos_recurrencia.rename(columns={'site_id' : 'usuarios.siteID'})
    
    datos_recurrencia['nombreDepartamento'] = datos_recurrencia['usuarios.nombreDepartamento']
    datos_recurrencia['nombreMunicipio'] = datos_recurrencia['usuarios.nombreMunicipio']
    datos_recurrencia['idBeneficiario'] = datos_recurrencia['usuarios.idBeneficiario']
    datos_recurrencia['fecha'] = datos_recurrencia['usuarios.fecha']
    datos_recurrencia['anyo'] = datos_recurrencia['usuarios.anyo']
    datos_recurrencia['mes'] = datos_recurrencia['usuarios.mes']
    datos_recurrencia['dia'] = datos_recurrencia['usuarios.dia']
except:
    pass

# Insertando recurrencia de usuario en indice principal

la lista use_these_keys se usa para referenciar cuales campos del dataframe irán al indice final. si los datos no se declaran en este, no se insertarán

In [82]:
try:
    use_these_keys = ['usuarios.nomCentroDigital'
                      , 'usuarios.codISO'
                      , 'usuarios.idBeneficiario'
                      , 'usuarios.localidad'
                      , 'usuarios.siteID'
                      , 'usuarios.nombreDepartamento'
                      , 'usuarios.sistemaEnergia'
                      , 'usuarios.nombreMunicipio'
                      , 'usuarios.location'
                      , 'usuarios.usuariosNuevos'
                      , 'usuarios.totales.usuariosNuevos'
                      , 'usuarios.fechaControl'
                      , 'usuarios.fecha'
                      , 'usuarios.anyo'
                      , 'usuarios.mes'
                      , 'usuarios.dia'
                      , 'usuarios.hora'
                      , 'usuarios.minuto'
                        , 'nombreDepartamento'
                        , 'nombreMunicipio'
                        , 'idBeneficiario'
                        , 'fecha'
                        , 'anyo'
                        , 'mes'
                        , 'dia'
                      , '@timestamp']

    datos_recurrencia['@timestamp'] = now.isoformat()
    def doc_generator(df):
        df_iter = df.iterrows()
        for index, document in df_iter:
            yield {
                    "_index": indice, 
                    "_id": f"{ 'Recurrencia-' + str(document['usuarios.siteID']) + '-' + str(document['usuarios.fechaControl'])+ str(random.randrange(1000))}",
                    "_source": filterKeys(document),
                }
    salida = helpers.bulk(es, doc_generator(datos_recurrencia))
    print("Fecha: ", now,"- recurrencia de usuario a indice:",salida[0])
except:
    print("Fecha: ", now,"- Ninguna recurrencia de usuario para insertar en indice principal")

Fecha:  2021-08-17 22:13:38.600373 - recurrencia de usuario a indice: 1


## Insertando detalle MACs

In [83]:
use_these_keys = ['usuarios.siteID'
                  , 'usuarios.nomCentroDigital'
                  , 'usuarios.codISO'
                  , 'usuarios.localidad'
                  , 'usuarios.nombreDepartamento'
                  , 'usuarios.sistemaEnergia'
                  , 'usuarios.nombreMunicipio'
                  , 'usuarios.idBeneficiario'
                  , 'usuarios.location'
                  , 'usuarios.macRed'
                  , 'usuarios.apGroup'
                  , 'usuarios.dispositivo.mac'
                  , 'usuarios.dispositivo.tipo'
                  , 'usuarios.dispositivo.nombre'
                  , 'usuarios.dispositivo.marca'
                  , 'usuarios.dispositivo.tecnologia'
                  , 'usuarios.dispositivo.sisOperativo'
                  , 'usuarios.fechaControl'
                  , 'usuarios.fecha'
                  , 'usuarios.anyo'
                  , 'usuarios.mes'
                  , 'usuarios.dia'
                  , 'usuarios.hora'
                  , 'usuarios.minuto'
                  , '@timestamp']
def doc_generator_mac(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{ 'detalleMAC-' + str(document['usuarios.siteID']) + '-' + str(document['usuarios.fechaControl']) + '-' + str(document['usuarios.dispositivo.mac']) + str(random.randrange(1000))  }",
                "_source": filterKeys(document),
            }

In [84]:
try:
    dispositivoUsuarios = datos_det_conex[['site_id','usuarios.macRed','usuarios.apGroup'
                                ,'mac_usuario','fecha_control'
                                ,'usuarios.tipoDispositivoUsuarios'
                                ,'name','manufacturer','radio.band'
                                ,'usuarios.sistemaOperativoUsuarios']].drop_duplicates(subset=['mac_usuario'])
    dispositivoUsuarios.fillna({'usuarios.tipoDispositivoUsuarios':'No identificado'
                               ,'name':'No identificado'
                               ,'manufacturer':'No identificado'
                               ,'radio.band':'No identificado'
                               ,'usuarios.sistemaOperativoUsuarios':'No identificado'
                               ,'usuarios.macRed':'No identificado'
                               ,'usuarios.apGroup':'No identificado'
                               },inplace=True)

    dispositivoUsuarios = pd.merge(dispositivoUsuarios, datos_semilla, on='site_id',how='inner')
    dispositivoUsuarios= dispositivoUsuarios.rename(columns={
                                                     'site_id' : 'usuarios.siteID'
                                                    ,'mac_usuario' : 'usuarios.dispositivo.mac'
                                                    ,'usuarios.tipoDispositivoUsuarios' : 'usuarios.dispositivo.tipo'
                                                    ,'name' : 'usuarios.dispositivo.nombre'
                                                    ,'manufacturer' : 'usuarios.dispositivo.marca'
                                                    ,'radio.band' : 'usuarios.dispositivo.tecnologia'
                                                    ,'usuarios.sistemaOperativoUsuarios' : 'usuarios.dispositivo.sisOperativo'
                                                    ,'fecha_control':'usuarios.fechaControl'
                                                    })
    dispositivoUsuarios["usuarios.fecha"] = dispositivoUsuarios["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    dispositivoUsuarios["usuarios.anyo"] = dispositivoUsuarios["usuarios.fecha"].str[0:4]
    dispositivoUsuarios["usuarios.mes"] = dispositivoUsuarios["usuarios.fecha"].str[5:7]
    dispositivoUsuarios["usuarios.dia"] = dispositivoUsuarios["usuarios.fecha"].str[8:10]
    dispositivoUsuarios["usuarios.hora"] = dispositivoUsuarios["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    dispositivoUsuarios["usuarios.minuto"] = dispositivoUsuarios["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    dispositivoUsuarios['@timestamp'] = now.isoformat()
    salida = helpers.bulk(es, doc_generator_mac(dispositivoUsuarios))
    print("Fecha: ", now,"- Detalle MACs a indice:",salida[0])
except:
    print("Fecha: ", now,"- Ningún Detalle MACs para insertar en indice principal")

Fecha:  2021-08-17 22:13:38.600373 - Detalle MACs a indice: 9


## Escribiendo en indice la información de tiempo promedio sesión en sitio

In [85]:
use_these_keys = ['usuarios.fecha'
                  , 'usuarios.siteID'
                  ,'usuarios.tiempoPromedioSesionSitio'
                  , 'usuarios.nomCentroDigital'
                  , 'usuarios.codISO'
                  , 'usuarios.idBeneficiario'
                  , 'usuarios.localidad'
                  , 'usuarios.nombreDepartamento'
                  , 'usuarios.sistemaEnergia'
                  , 'usuarios.nombreMunicipio'
                  , 'usuarios.location'
                  , 'usuarios.macRed'
                  , 'usuarios.apGroup'
                  #, 'usuarios.usuariosConectados'
                  #, 'usuarios.sesiones_Usuarios'
                  , 'usuarios.fechaControl'
                  , 'usuarios.anyo'
                  , 'usuarios.mes'
                  , 'usuarios.dia'
                  , 'usuarios.hora'
                  , 'usuarios.minuto'
                    , 'nombreDepartamento'
                    , 'nombreMunicipio'
                    , 'idBeneficiario'
                    , 'fecha'
                    , 'anyo'
                    , 'mes'
                    , 'dia'
                  , '@timestamp']
def doc_generator(df):
        df_iter = df.iterrows()
        for index, document in df_iter:
            document = document.rename(index={'usuarios.tiempoPromedioSesionSitio_x': 'usuarios.tiempoPromedioSesionSitio','usuarios.macRed_x':'usuarios.macRed','usuarios.apGroup_x':'usuarios.apGroup','usuarios.fechaControl_x':'usuarios.fechaControl'})
            yield {
                    "_index": indice, 
                    "_id": f"{str(document['usuarios.siteID']) + '-' + str(document['usuarios.fechaControl']) + '-' + str(document['usuarios.macRed'])+'-'+str(random.randrange(10000000))}",
                    "_source": filterKeys(document),
                }
            

In [86]:
use_these_keys

['usuarios.fecha',
 'usuarios.siteID',
 'usuarios.tiempoPromedioSesionSitio',
 'usuarios.nomCentroDigital',
 'usuarios.codISO',
 'usuarios.idBeneficiario',
 'usuarios.localidad',
 'usuarios.nombreDepartamento',
 'usuarios.sistemaEnergia',
 'usuarios.nombreMunicipio',
 'usuarios.location',
 'usuarios.macRed',
 'usuarios.apGroup',
 'usuarios.fechaControl',
 'usuarios.anyo',
 'usuarios.mes',
 'usuarios.dia',
 'usuarios.hora',
 'usuarios.minuto',
 'nombreDepartamento',
 'nombreMunicipio',
 'idBeneficiario',
 'fecha',
 'anyo',
 'mes',
 'dia',
 '@timestamp']

Se agrupa por lugar_cod, fecha_control, mac_ap, mac_usuario y se promedia el tiempo_sesion_minutos. Este genera el campo del indice final:

usuarios.tiempoPromedioSesionSitio

In [87]:
if  continuar:

    try:

        datos_consumos = datos_consumos.rename(columns={'lugar_cod' :'site_id'})
        datos_consumos['mac_ap'] = datos_consumos['mac_ap'].str.replace('-',':')
        datos_consumos['fecha_control'] = datos_consumos["fecha_inicio"].str[0:-4] + '0:00'
        tiempoPromedioSesionSitio=datos_consumos[['site_id','mac_ap','fecha_control','tiempo_sesion_minutos']].groupby(['site_id','mac_ap','fecha_control']).agg(['mean']).reset_index()
        tiempoPromedioSesionSitio.columns = tiempoPromedioSesionSitio.columns.droplevel(1)
        tiempoPromedioSesionSitio = tiempoPromedioSesionSitio.rename(columns={'tiempo_sesion_minutos' : 'usuarios.tiempoPromedioSesionSitio'})
        tiempoPromedioSesionSitio['usuarios.tiempoPromedioSesionSitio'] = round(tiempoPromedioSesionSitio['usuarios.tiempoPromedioSesionSitio'],6)
        tiempoPromedioSesionSitio = tiempoPromedioSesionSitio.rename(columns={'mac_ap' : 'usuarios.macRed'
                                                                              ,'fecha_control' : 'usuarios.fechaControl'})
        tiempoPromedioSesionSitio = pd.merge(tiempoPromedioSesionSitio,datos_semilla, on ='site_id', how='inner')
        tiempoPromedioSesionSitio = pd.merge(tiempoPromedioSesionSitio, datos_dev, on=['site_id','usuarios.macRed'], how='left')
        tiempoPromedioSesionSitio.fillna({'usuarios.apGroup':'No identificado'},inplace=True)

        ##################
        tiempoPromedioSesionSitio = tiempoPromedioSesionSitio.rename(columns={'site_id' : 'usuarios.siteID'})
        try:
            tiempoPromedioSesionSitio["usuarios.fecha"] = tiempoPromedioSesionSitio["usuarios.fechaControl"].str[0:10]
        except:
            tiempoPromedioSesionSitio["usuarios.fecha"] = ""

        try:
            tiempoPromedioSesionSitio["usuarios.anyo"] = tiempoPromedioSesionSitio["usuarios.fecha"].str[0:4]
        except:
            tiempoPromedioSesionSitio["usuarios.anyo"] = ""

        try:
            tiempoPromedioSesionSitio["usuarios.mes"] = tiempoPromedioSesionSitio["usuarios.fecha"].str[5:7]
        except:
            tiempoPromedioSesionSitio["usuarios.mes"] = ""

        try:
            tiempoPromedioSesionSitio["usuarios.dia"] = tiempoPromedioSesionSitio["usuarios.fecha"].str[8:10]
        except:
            tiempoPromedioSesionSitio["usuarios.dia"] = ""

        try:
            tiempoPromedioSesionSitio["usuarios.hora"] = tiempoPromedioSesionSitio["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]["usuarios.hora"] = tiempoPromedioSesionSitio["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
        except:
            tiempoPromedioSesionSitio["usuarios.hora"] = ""

        try:
            tiempoPromedioSesionSitio["usuarios.minuto"] = tiempoPromedioSesionSitio["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
        except:
            tiempoPromedioSesionSitio["usuarios.minuto"] = ""

        tiempoPromedioSesionSitio['nombreDepartamento'] = tiempoPromedioSesionSitio['usuarios.nombreDepartamento']
        tiempoPromedioSesionSitio['nombreMunicipio'] = tiempoPromedioSesionSitio['usuarios.nombreMunicipio']
        tiempoPromedioSesionSitio['idBeneficiario'] = tiempoPromedioSesionSitio['usuarios.idBeneficiario']
        tiempoPromedioSesionSitio['fecha'] = tiempoPromedioSesionSitio['usuarios.fecha']
        tiempoPromedioSesionSitio['anyo'] = tiempoPromedioSesionSitio['usuarios.anyo']
        tiempoPromedioSesionSitio['mes'] = tiempoPromedioSesionSitio['usuarios.mes']
        tiempoPromedioSesionSitio['dia'] = tiempoPromedioSesionSitio['usuarios.dia']
        tiempoPromedioSesionSitio['@timestamp'] = now.isoformat()
        
        salida = helpers.bulk(es, doc_generator(tiempoPromedioSesionSitio))

        print("Fecha: ", now,"- Tiempo promedio sesion en sitio insertado en indice principal:",salida[0])

    except Exception as e:
        print(e)
        print("Fecha: ", now,"- Nada para insertar en indice principal")

Fecha:  2021-08-17 22:13:38.600373 - Tiempo promedio sesion en sitio insertado en indice principal: 33816


In [89]:
try:
    datos_speed = datos_speed.drop(datos_speed[(datos_speed["result_download_mbps"]<0) | (datos_speed["result_upload_mbps"]<0)].index)
    datos_speed['trafico.totales.fecha'] = datos_speed['result_start_date'].str.split("T", n = 1, expand = True)[0]
    datos_speed['result_download_mbps'] = datos_speed['result_download_mbps'] * 1000
    datos_speed['result_upload_mbps'] = datos_speed['result_upload_mbps'] * 1000
    datos_speed = datos_speed[['beneficiary_code','trafico.totales.fecha','result_download_mbps','result_upload_mbps']].groupby(['beneficiary_code','trafico.totales.fecha']).agg(['max']).reset_index()
    datos_speed.columns = datos_speed.columns.droplevel(1)
    datos_speed.rename(columns={'result_download_mbps': 'trafico.anchoBandaDescarga'
                             ,'result_upload_mbps' :  'trafico.anchoBandaCarga'
                             , 'beneficiary_code' : 'site_id'
                             }, inplace=True)

    datos_speed["trafico.totales.anyo"] = datos_speed["trafico.totales.fecha"].str[0:4]
    datos_speed["trafico.totales.mes"] = datos_speed["trafico.totales.fecha"].str[5:7]
    datos_speed["trafico.totales.dia"] = datos_speed["trafico.totales.fecha"].str[8:10]
    datos_speed = pd.merge(datos_speed,  datos_semilla, on='site_id', how='inner')
    datos_speed = datos_speed.rename(columns={'site_id' : 'trafico.siteID'})
    datos_speed.dropna(subset=['trafico.anchoBandaDescarga','trafico.anchoBandaCarga'])
    datos_speed.fillna({'trafico.anchoBandaDescarga':0
                      , 'trafico.anchoBandaCarga':0
                       },inplace=True)
    datos_speed.fillna('', inplace=True)
    datos_speed['nombreDepartamento'] = datos_speed['trafico.nombreDepartamento']
    datos_speed['nombreMunicipio'] = datos_speed['trafico.nombreMunicipio']
    datos_speed['idBeneficiario'] = datos_speed['trafico.idBeneficiario']
    datos_speed['fecha'] = datos_speed['trafico.totales.fecha']
    datos_speed['anyo'] = datos_speed['trafico.totales.anyo']
    datos_speed['mes'] = datos_speed['trafico.totales.mes']
    datos_speed['dia'] = datos_speed['trafico.totales.dia']
    datos_speed['@timestamp'] = now.isoformat()

#    salida = helpers.bulk(es, doc_generator(datos_speed))
#    print("Fecha: ", now,"- Datos Velocidad carga y descarga (Trafico) en indice principal:",salida[0])
except:
    print("Fecha: ", now,"- No hay Datos Velocidad carga y descarga(Trafico) para insertar en indice principal")

Fecha:  2021-08-17 22:13:38.600373 - No hay Datos Velocidad carga y descarga(Trafico) para insertar en indice principal


In [92]:
datos_speed = datos_speed.drop(datos_speed[(datos_speed["result_download_mbps"]<0) | (datos_speed["result_upload_mbps"]<0)].index)
datos_speed['trafico.totales.fecha'] = datos_speed['result_start_date'].str.split("T", n = 1, expand = True)[0]
datos_speed['result_download_mbps'] = datos_speed['result_download_mbps'] * 1000
datos_speed['result_upload_mbps'] = datos_speed['result_upload_mbps'] * 1000
datos_speed = datos_speed[['beneficiary_code','trafico.totales.fecha','result_download_mbps','result_upload_mbps']].groupby(['beneficiary_code','trafico.totales.fecha']).agg(['max']).reset_index()
datos_speed.columns = datos_speed.columns.droplevel(1)
datos_speed.rename(columns={'result_download_mbps': 'trafico.anchoBandaDescarga'
                         ,'result_upload_mbps' :  'trafico.anchoBandaCarga'
                         , 'beneficiary_code' : 'site_id'
                         }, inplace=True)

datos_speed["trafico.totales.anyo"] = datos_speed["trafico.totales.fecha"].str[0:4]
datos_speed["trafico.totales.mes"] = datos_speed["trafico.totales.fecha"].str[5:7]
datos_speed["trafico.totales.dia"] = datos_speed["trafico.totales.fecha"].str[8:10]
datos_speed = pd.merge(datos_speed,  datos_semilla, on='site_id', how='inner')
datos_speed = datos_speed.rename(columns={'site_id' : 'trafico.siteID'})
datos_speed.dropna(subset=['trafico.anchoBandaDescarga','trafico.anchoBandaCarga'])
datos_speed.fillna({'trafico.anchoBandaDescarga':0
                  , 'trafico.anchoBandaCarga':0
                   },inplace=True)
datos_speed.fillna('', inplace=True)
datos_speed['nombreDepartamento'] = datos_speed['trafico.nombreDepartamento']
datos_speed['nombreMunicipio'] = datos_speed['trafico.nombreMunicipio']
datos_speed['idBeneficiario'] = datos_speed['trafico.idBeneficiario']
datos_speed['fecha'] = datos_speed['trafico.totales.fecha']
datos_speed['anyo'] = datos_speed['trafico.totales.anyo']
datos_speed['mes'] = datos_speed['trafico.totales.mes']
datos_speed['dia'] = datos_speed['trafico.totales.dia']
datos_speed['@timestamp'] = now.isoformat()

KeyError: 'result_download_mbps'

In [90]:
try:
    datos_speed.rename(columns={
        'trafico.siteID':'usuarios.siteID',
        'trafico.nombreDepartamento':'usuarios.nombreDepartamento',
        'trafico.codISO':'usuarios.codISO',
        'trafico.sistemaEnergia':'usuarios.sistemaEnergia',
        'trafico.nombreMunicipio':'usuarios.nombreMunicipio',
        'trafico.localidad' : 'usuarios.localidad',
        'trafico.nomCentroDigital' : 'usuarios.nomCentroDigital',
        'trafico.idBeneficiario':'usuarios.idBeneficiario',
        'trafico.location':'usuarios.location',
        'trafico.anchoBandaDescarga':'usuarios.anchoBandaDescarga',
        'trafico.anchoBandaCarga':'usuarios.anchoBandaCarga',
        'trafico.totales.fecha':'usuarios.fecha',
        'trafico.totales.anyo':'usuarios.anyo',
        'trafico.totales.mes':'usuarios.mes',
        'trafico.totales.dia':'usuarios.dia' }, inplace=True)
    use_these_keys = ['usuarios.nomCentroDigital'
                  , 'usuarios.codISO'
                  , 'usuarios.idBeneficiario'
                  , 'usuarios.localidad'
                  , 'usuarios.siteID'
                  , 'usuarios.nombreDepartamento'
                  , 'usuarios.sistemaEnergia'
                  , 'usuarios.nombreMunicipio'
                  , 'usuarios.localidad'    
                  , 'usuarios.nomCentroDigital'    
                  , 'usuarios.location'
                  , 'usuarios.fecha'
                  , 'usuarios.anchoBandaDescarga'
                  , 'usuarios.anchoBandaCarga'
                  , 'usuarios.anyo'
                  , 'usuarios.mes'
                  , 'usuarios.dia'
                  , 'nombreDepartamento'
                  , 'nombreMunicipio'
                  , 'idBeneficiario'
                  , 'fecha'
                  , 'anyo'
                  , 'mes'
                  , 'dia'    
                  , '@timestamp']

    def doc_generator_u(df):
        df_iter = df.iterrows()
        for index, document in df_iter:
            yield {
                    "_index": indice, 
                    "_id": f"{'VelUsu-'+ str(document['usuarios.siteID']) + '-' + str(document['usuarios.fecha'])}",
                    "_source": filterKeys(document)
                }
    salida = helpers.bulk(es, doc_generator_u(datos_speed))
    print("Fecha: ", now,"- Datos Velocidad carga y descarga(Usuario) en indice principal:",salida[0])        
except:
    print("Fecha: ", now,"- No hay Datos Velocidad carga y descarga(Usuario) para insertar en indice principal")    

Fecha:  2021-08-17 22:13:38.600373 - Datos Velocidad carga y descarga(Usuario) en indice principal: 0


Se descartan las respuestas Si y No de las valoraciones, las cuales corresponden a la pregunta: "Te gustaría calificar tu última conexión en". De esta forma solo se dejan las respuestas asociadas a la percepción de calidad por parte del usuario.
Se estandariza lugar_cod a site_id
Se estadariza fecha para agrupar (solo se toma yyyy-mm-dd)
Se Calcula nivel de valoracion por pregunta y total de valoraciones
Para cada centro de conexión, pregunta, se contabilizan las valoraciones. El calculo es diario

In [91]:
try:
    datos_valoraciones = datos_valoraciones.drop(datos_valoraciones[(datos_valoraciones["respuesta"].isin(['Si','No']))].index)
    #datos_valoraciones['mac_usuario'] = datos_valoraciones['mac_usuario'].str.replace('-',':')
    datos_valoraciones = datos_valoraciones.rename(columns={'lugar_cod': 'site_id'})
    datos_valoraciones['fecha'] = datos_valoraciones["fechahora"].str.split(" ", n = 1, expand = True)[0]
    nivel_valoraciones=datos_valoraciones[['fechahora', 'site_id'
                                         , 'respuesta'
                                         , 'fecha']].groupby(['site_id','respuesta','fecha']).agg(['count']).reset_index()
    nivel_valoraciones.columns = nivel_valoraciones.columns.droplevel(1)
    nivel_valoraciones = nivel_valoraciones.rename(columns={'fechahora' :'usuarios.tablero13.nivelesSatisfaccionUsuarios'
                                                           ,'respuesta' :'usuarios.tablero13.nivelesSatisfaccionUsuariosRespuesta'
                                                           ,'fecha' : 'usuarios.tablero13.fechaCalificacion'})
    
    total_valoraciones = datos_valoraciones[['site_id','fechahora','fecha']].groupby(['site_id','fecha']).agg(['count']).reset_index()
    total_valoraciones.columns = total_valoraciones.columns.droplevel(1)
    total_valoraciones = total_valoraciones.rename(columns={'fechahora' :'usuarios.tablero13.cantidadCalificaciones'
                                                           ,'fecha' : 'usuarios.tablero13.fechaCalificacion'})

    #Con ambos se calcula el porcentaje de cada respuesta
    nivel_valoraciones =  pd.merge(nivel_valoraciones,total_valoraciones, on=['site_id','usuarios.tablero13.fechaCalificacion'],how='inner')
    nivel_valoraciones['usuarios.tablero13.nivelesSatisfaccionUsuariosRespuestaPorCien'] = ((nivel_valoraciones['usuarios.tablero13.nivelesSatisfaccionUsuarios']) / nivel_valoraciones['usuarios.tablero13.cantidadCalificaciones']).round(4)
except:
    total_valoraciones = pd.DataFrame(columns=['site_id','usuarios.tablero13.fechaCalificacion'
                                              ,'usuarios.tablero13.cantidadCalificaciones'])
    nivel_valoraciones = pd.DataFrame(columns=['site_id','usuarios.tablero13.fechaCalificacion'
                                              ,'usuarios.tablero13.nivelesSatisfaccionUsuarios'
                                              ,'usuarios.tablero13.nivelesSatisfaccionUsuariosRespuesta'
                                              ,'usuarios.tablero13.nivelesSatisfaccionUsuariosRespuestaPorCien'])
    

## Cruzando con semilla las agregaciones de valoraciones

In [ ]:
mintic_valoraciones = pd.merge(datos_semilla, nivel_valoraciones, on='site_id',how='inner')

In [ ]:
mintic_valoraciones["usuarios.tablero13.nivelesSatisfaccionUsuarios"].sum(axis=0)

In [ ]:
mintic_valoraciones

## Escribiendo en indice la información de Valoraciones

Se convierten los nulos a ceros a nivelesSatisfaccionUsuarios y cantidadCalificaciones

In [ ]:
try:
    mintic_valoraciones.fillna({'usuarios.tablero13.nivelesSatisfaccionUsuarios':0
                               ,'usuarios.tablero13.nivelesSatisfaccionUsuariosRespuestaPorCien':0},inplace=True)
    mintic_valoraciones[['usuarios.tablero13.nivelesSatisfaccionUsuarios']] = mintic_valoraciones[['usuarios.tablero13.nivelesSatisfaccionUsuarios']].astype(int)
    mintic_valoraciones = mintic_valoraciones.rename(columns={'site_id' : 'usuarios.tablero13.siteID'})
    mintic_valoraciones.dropna(subset=['usuarios.tablero13.nivelesSatisfaccionUsuariosRespuesta'], inplace=True)
    mintic_valoraciones["usuarios.tablero13.anyo"] = mintic_valoraciones["usuarios.tablero13.fechaCalificacion"].str[0:4]
    mintic_valoraciones["usuarios.tablero13.mes"] = mintic_valoraciones["usuarios.tablero13.fechaCalificacion"].str[5:7]
    mintic_valoraciones["usuarios.tablero13.dia"] = mintic_valoraciones["usuarios.tablero13.fechaCalificacion"].str[8:10]
    mintic_valoraciones['nombreDepartamento'] = mintic_valoraciones['usuarios.tablero13.nombreDepartamento']
    mintic_valoraciones['nombreMunicipio'] = mintic_valoraciones['usuarios.tablero13.nombreMunicipio']
    mintic_valoraciones['idBeneficiario'] = mintic_valoraciones['idBeneficiario']
    mintic_valoraciones['fecha'] = mintic_valoraciones['usuarios.tablero13.fechaCalificacion']
    mintic_valoraciones['anyo'] = mintic_valoraciones['usuarios.tablero13.anyo']
    mintic_valoraciones['mes'] = mintic_valoraciones['usuarios.tablero13.mes']
    mintic_valoraciones['dia'] = mintic_valoraciones['usuarios.tablero13.dia']
except:
    
    print('Null')
    
    pass

In [ ]:
use_these_keys = ['usuarios.tablero13.nomCentroDigital'
                  , 'usuarios.tablero13.codISO'
                  , 'usuarios.tablero13.localidad'
                  , 'usuarios.tablero13.siteID'
                  , 'usuarios.tablero13.nombreDepartamento'
                  , 'usuarios.tablero13.sistemaEnergia'
                  , 'usuarios.tablero13.nombreMunicipio'
                  , 'idBeneficiario'
                  , 'usuarios.tablero13.location'
                  , 'usuarios.tablero13.nivelesSatisfaccionUsuariosRespuesta'
                  , 'usuarios.tablero13.fechaCalificacion'
                  , 'usuarios.tablero13.nivelesSatisfaccionUsuarios'
                  , 'usuarios.tablero13.nivelesSatisfaccionUsuariosRespuestaPorCien'               
                  , 'usuarios.tablero13.anyo'
                  , 'usuarios.tablero13.mes'
                  , 'usuarios.tablero13.dia'
                    , 'nombreDepartamento'
                    , 'nombreMunicipio'
                    , 'idBeneficiario'
                    , 'fecha'
                    , 'anyo'
                    , 'mes'
                    , 'dia'
                  , '@timestamp']
try:
    mintic_valoraciones['@timestamp'] = now.isoformat()
    def doc_generator(df):
        df_iter = df.iterrows()
        for index, document in df_iter:
            yield {
                    "_index": indice, 
                    "_id": f"{'Valoracion-' + str(document['usuarios.tablero13.siteID']) + '-' + str(document['usuarios.tablero13.fechaCalificacion']) + '-' + str(document['usuarios.tablero13.nivelesSatisfaccionUsuariosRespuesta']) + '-' +str(random.randrange(10000))}",
                    "_source": filterKeys(document),
                }
    salida = helpers.bulk(es, doc_generator(mintic_valoraciones))
    print("Fecha: ", now,"- Valoraciones insertadas en indice principal:",salida[0])
except:
    print("Fecha: ", now,"- No se insertaron valoraciones en indice principal")

## Insertando total de calificaciones

In [ ]:
mintic_calificaciones = pd.merge(datos_semilla,  total_valoraciones, on='site_id', how='inner')

In [ ]:
try:
    mintic_calificaciones.fillna({'usuarios.tablero13.cantidadCalificaciones':0},inplace=True)
    mintic_calificaciones[['usuarios.tablero13.cantidadCalificaciones']] = mintic_calificaciones[['usuarios.tablero13.cantidadCalificaciones']].astype(int)
    mintic_calificaciones = mintic_calificaciones.rename(columns={'site_id' : 'usuarios.siteID'})
    mintic_calificaciones.dropna(subset=['usuarios.tablero13.cantidadCalificaciones'], inplace=True)
    mintic_calificaciones["usuarios.tablero13.anyo"] = mintic_calificaciones["usuarios.tablero13.fechaCalificacion"].str[0:4]
    mintic_calificaciones["usuarios.tablero13.mes"] = mintic_calificaciones["usuarios.tablero13.fechaCalificacion"].str[5:7]
    mintic_calificaciones["usuarios.tablero13.dia"] = mintic_calificaciones["usuarios.tablero13.fechaCalificacion"].str[8:10]
    mintic_calificaciones["usuarios.tablero13.fecha"] = mintic_calificaciones["usuarios.tablero13.fechaCalificacion"]
    mintic_calificaciones['nombreDepartamento'] = mintic_calificaciones['usuarios.tablero13.nombreDepartamento']
    mintic_calificaciones['nombreMunicipio'] = mintic_calificaciones['usuarios.tablero13.nombreMunicipio']
    mintic_calificaciones['idBeneficiario'] = mintic_calificaciones['usuarios.idBeneficiario']
    mintic_calificaciones['fecha'] = mintic_calificaciones['usuarios.tablero13.fechaCalificacion']
    mintic_calificaciones['anyo'] = mintic_calificaciones['usuarios.tablero13.anyo']
    mintic_calificaciones['mes'] = mintic_calificaciones['usuarios.tablero13.mes']
    mintic_calificaciones['dia'] = mintic_calificaciones['usuarios.tablero13.dia']
except:

    pass

In [ ]:
use_these_keys = ['usuarios.tablero13.nomCentroDigital'
                  , 'usuarios.tablero13.codISO'
                  , 'idBeneficiario'
                  , 'usuarios.tablero13.localidad'
                  , 'usuarios.siteID'
                  , 'usuarios.tablero13.nombreDepartamento'
                  , 'usuarios.tablero13.sistemaEnergia'
                  , 'usuarios.tablero13.nombreMunicipio'
                  , 'usuarios.tablero13.location'
                  , 'usuarios.tablero13.cantidadCalificaciones'
                  , 'usuarios.tablero13.fechaCalificacion'
                  , 'usuarios.tablero13.fecha'
                  , 'usuarios.tablero13.anyo'
                  , 'usuarios.tablero13.mes'
                  , 'usuarios.tablero13.dia'
                    , 'nombreDepartamento'
                    , 'nombreMunicipio'
                    , '@timestamp']
try:
    mintic_calificaciones['@timestamp'] = now.isoformat()
    def doc_generator(df):
        df_iter = df.iterrows()
        for index, document in df_iter:
            yield {
                    "_index": indice, 
                    "_id": f"{ 'Calificacion-' + str(document['usuarios.siteID']) + '-' + str(document['usuarios.tablero13.fechaCalificacion']) + '-' +str(random.randrange(10000))}",
                    "_source": filterKeys(document),
                }
    salida = helpers.bulk(es, doc_generator(mintic_calificaciones))
    print("Fecha: ", now,"- Total calificaciones insertadas en indice principal:",salida[0])
except:
    print("Fecha: ", now,"- No se insertaron totales de calificaciones en indice principal")

### Guardando fecha para control de ejecución

* Se actualiza la fecha de control. Si el calculo supera la fecha hora actual, se asocia esta ultima.

In [ ]:
fecha_ejecucion= (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")[0:15] + '0:00'    

if fecha_ejecucion > str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00':
    fecha_ejecucion = str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00'
response = es.index(
        index = indice_control,
        id = 'jerarquia-tablero12',
        body = { 'jerarquia-tablero12': 'jerarquia-tablero12','usuarios.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)